<a href="https://colab.research.google.com/github/samueldatasci/Modelling_biosystems/blob/main/Assignment_20241212.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TO CHECK**<br>
* Function dataloader and model_generator calculate max_lag differently; which one is correct?
* Carefully check function model_generator and/or try to get an alternative;

In [ ]:
#number of poles of a system is called the order of the system - https://electronics.stackexchange.com/questions/451128/what-does-dominant-pole-pole-mean-related-to-capacitors-and-filters-or-electroni
#TC -> "This "impulse response" is made up of different components, each of which decays with a different time constant. Every pole contributes a different component with a different time constant. The time constant is a measure of how fast a component decays: the bigger the time constant, the slower the component decay
#A dominant pole is a pole whose time constant is much "slower", i.e. bigger, than all the other time constants of the circuit, therefore the corresponding component is still observable after all the other, faster decaying, components have died off.

!pip install slycot
!pip install control
!pip install pymannkendall

In [ ]:
from google.colab import files
import seaborn as sns
import io
import math
import pandas as pd
import numpy as np
import pymannkendall as mk
import statsmodels.api as sm
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import control as ctrl
#from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
from scipy import signal


In [ ]:
#global vars

g_subtract_min = True
g_normalize = True


g_upload = False

g_r2_dictionary = {}


In [ ]:
def dataloader(data, na, nb, d):

    # Initialize empty arrays for lagged features
    y = data['y']
    u = data['u']
    X = pd.DataFrame()


    # Create lagged features for output ('a' parameters)
    for i in range(1,na+1):
        X['y-{}'.format(i)] = y.shift(i)
    # Create lagged features for input ('b' parameters)
    for i in range(0,nb):
        X['u-{}'.format(i + d)] = u.shift(i + d)


    #Only select data after a certain value
    max_lag = max(na,nb+d)


    X = X.iloc[max_lag:]
    # Set the target values
    y = y[max_lag:]

    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    return X, y



## NOT USED
def r_2(y, y_hat):  ## NOT USED
  rskl = r2_score(y,y_hat) # (y_true, y_pred)
  return rskl
#   y_mean = np.mean(y)
#   y_hat_mean = np.mean(y_hat)
#   sst = np.sum((y - y_mean) ** 2)
#   #ssr = np.sum((y - y_hat_mean) ** 2)
#   ssr = np.sum((y - y_hat) ** 2)
#   r_squared = 1 - (ssr / sst)
#   #print("SSR:", ssr)
#   #print("SST:", sst)
#   #print("Mean of y:", y_mean)
#   #print("Variance of y:", np.var(y))
#   print(f'Mine:{r_squared} Sklearn: {rskl}')
#   return r_squared
  ## NOT USED



def model_metrics(y_real,y_sim,na,nb):
        # Calculate r-squared and AIC and RMSE and BIC and FPE
        k = na + nb # number of parameters
        n = len(y_real)
        SSR = np.sum((y_real - y_sim) ** 2)
        #Calculate usefull Model evaluation parameters
        r_squared = r_2(y_real, y_sim)
        aic =  2 * (na + nb) + n * np.log(1 - r_squared) # Evaluates the complexicity of the Model
        Rmse = rmse(y_real, y_sim[0:])  #Root Mean Squared Error (Less sensitive to outliers: It is less affected by extreme errors compared to RMSE)
        fpe = SSR * (n + k) / (n - k)                  #Final Prediction Error(used for model order selection)(Considers both model fit and complexity) (Asymptotically equivalent to AIC: For large sample sizes, FPE and AIC tend to select the same model)
        bic = n * np.log(SSR / n) + k * np.log(n)      #Bayesian Information Criterion (BIC penalizes model complexity more heavily than AIC.)(Consistent: more likely to select the true model when sample is bigger)
        # YIC calculation
        yic = n * np.log(SSR / n) + k * (np.log(n) + np.log(np.log(n))) #NOT THE SLIDES FORMULA (Gemini)


        return r_squared, aic, Rmse, bic, fpe

def quadratic_eq( a, b, c):

    # calculating discriminant using formula
    dis = b * b - 4 * a * c
    sqrt_val = math.sqrt(abs(dis))

    # checking condition for discriminant
    if dis > 0:
        #print("real and different roots")
        root1 = (-b + sqrt_val)/(2 * a)
        root2 = (-b - sqrt_val)/(2 * a)

    elif dis == 0:
        #print("real and same roots")
        roots = -b / (2 * a)
        root1 = roots
        root2 = roots

    # when discriminant is less than 0
    else:
        #print("Complex Roots")
        root1 = - b / (2 * a), + i, sqrt_val / (2 * a)
        root2 = - b / (2 * a), - i, sqrt_val / (2 * a)

    return root1, root2

In [ ]:
def model_generator(data, na, nb, d):

  #print(f"Athlete: {code_name[i]}")
  print('\nModel with {} a parameters, {} b parameters and {} time delay'.format(na,nb,d))
  #load the associated data using dataloader(data, na, nb, d)
  X_loader, y_loader = dataloader(data, na,  nb, d)

  ##Calculate the a and b parameters      #parameters vector [a0...an, b0...bm].T by the formula inv(X.T dot X) dot X.T dot y
  parameters = np.linalg.inv(X_loader.T.dot(X_loader)).dot(X_loader.T).dot(y_loader)
  parameters_a = parameters[:na]
  parameters_b = parameters[na:]

  #Create Transfer Function
  TF = ctrl.tf(parameters_b, [1, *[-x for x in parameters_a]]) #.tf(num,den)


  ## Simulate values of Y
  ysim = np.zeros(len(y_loader)) # the plus one is because the first value is initialized and not simulated
  max_lag= max(na,nb+d-1)  # Maximum lag across both endogenous and exogenous variables

  #Should I initialize the first values that can be measured?  this way is
  #ysim[0]= y_loader.values[0] # initialize the first value -> Predicted resut: [127,0,0,0,131,145]
  ysim[:max_lag] = y_loader.values[:max_lag]


  for k in range(max_lag, len(X_loader)):
  # Add the contribution from autoregressive terms (endogenous variables)
    for i_a in range(1,na+1):
        #print("parameter a: ",i_a)
        ysim[k] += parameters_a[i_a-1] * ysim[k - i_a]  # Note: parameters_a starts at index 0

  # Add the contribution from exogenous input terms (with delay d)
    for j_b in range(nb):
        ysim[k] += parameters_b[j_b] * X_loader['u-{}'.format(d+j_b)].iloc[k-j_b]  # Note: parameters_b starts at index 0
        #print('u-{}'.format(d+j_b),"  Parameter B:",j_b,"  Delay:",d)



  #Used? Needed?
  num = parameters_b # b [b1,b2,b3,...] (exogenous)
  den = [1, -parameters_a] #a [1,a2,a3,...] (endogenous)




  #Calculate SSG(K) and TC
  ##SSG = how much the output of a system changes in response to a change in the input.
  SSG = sum(parameters_b)/ (1 - sum(parameters_a)) #a unit change in the input (u) will result in a 1.33 unit change in the output (y)
  SSG2 = sum(num)/ (sum(den))   #print("SSG:", SSG,'SSG2:',SSG2)

  #Calculate the TC
  #TF_poles = ctrl.poles(TF) #ctrl.pole ???
  TF_zeros = ctrl.zeros(TF)
  TF_DCG = ctrl.dcgain(TF) ## SSG = DCgain ?????
  #TF_dominant_pole = min(TF_poles, key=lambda p: abs(p.real))  #The dominant pole is the one with the smallest magnitude real part (closest to the imaginary axis). This pole will largely determine the system’s time constant.

  TF_poles = ctrl.poles(TF)
  if not np.any(TF_poles):  # Check if TF_poles is empty || .any() -> at least one is not 0 || .all() -> all are not 0
      print("Warning: No poles found for the system. Skipping dominant pole calculation.")
      TF_dominant_pole = None  # Or assign a default value
      TF_TC = None  # Or assign a default value
  else:
      TF_dominant_pole = min(TF_poles, key=lambda p: abs(p.real))
      TF_TC = -1 / TF_dominant_pole.real

  TF_TC = -1 / TF_dominant_pole.real #The time constant ( \tau ) is the negative reciprocal of the real part of the dominant pole.








  print(f'SSG:{SSG}, SSG2:{SSG2}, DCG:{TF_DCG}')
  #Check if parameters_a has more than one element to avoid the error
  #if len(parameters_a) > 1:
  #    poles = 1/parameters_a[0]
  #else:
  #    poles = 1/parameters_a  # In case parameters_a has only one element
  if parameters_a.size > 0:  # Check if parameters_a is not empty
    poles = 1/parameters_a[0]
  else:
    poles = 1
  #print("parameters a", parameters_a, "Poles:", poles)
  poles2 = 1/den[1]
  #print("parameters a2", den, "Poles2:", poles2)

  #Check if poles is an array before calculating dominant_pole
  #if isinstance(poles, np.ndarray):
  #    dominant_pole = poles[np.argmin(np.abs(poles.imag))] #get the dominant
  #else:
  #   dominant_pole = poles  # If poles is a scalar, directly assign to dominant_pole
  dominant_pole = poles

  tau = -1 / np.log(dominant_pole)

  print(f"Tau:{tau}, TF_TC:{TF_TC}")
  #print("Time constant:", tau)
  #tau2 = -1 / np.log(poles[np.argmin(np.abs(poles.imag))])
  #print("Time constant2:", tau2)


  #Models Stability
  num = parameters_b # b [b1,b2,b3,...] (exogenous)
  den1 =  np.insert(-parameters_a, 0, 1)
  den = [1] + (-parameters_a).tolist() #a [1,a2,a3,...] (endogenous)
  #TF = ctrl.TransferFunction(num, den)    #define transfer function
#  poles, zeroes = ctrl.pzmap(TF, plot=True)  #PLOT CIRCLE AND MODEL (DELETE?CHANGE?)
  #add unit circle
  #plt.gca().add_patch(plt.Circle((0, 0), radius=1, fill=False, color='gray', linestyle='--'))
  #plt.show()
  #New Verions (TESTED, but NOT CONFIRMED TO DO THE SAME)
  #poles, zeroes = ctrl.pole_zero_map(TF)
  #ctrl.pole_zero_plot(TF)         #



  ## Model evaluation for each generated model use r_squared and aic def above
  r_squared, aic, Rmse, bic, fpe = model_metrics(y_loader,ysim,na,nb)

  return y_loader, ysim, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe


# 1º Practical assignment

In [ ]:
#g_upload = True
if g_upload:
    uploaded = files.upload()


In [ ]:
A1M1_df = pd.read_csv('MOB_Dataset1_A1_M1.csv', names=['y','u'])
A1M2_df = pd.read_csv('MOB_Dataset1_A1_M2.csv', names=['y','u'])
A2M1_df = pd.read_csv('MOB_Dataset1_A2_M3.csv', names=['y','u'])
A2M2_df = pd.read_csv('MOB_Dataset1_A2_M4.csv', names=['y','u'])
A3M1_df = pd.read_csv('MOB_Dataset1_A3_M5.csv', names=['y','u'])
A3M2_df = pd.read_csv('MOB_Dataset1_A3_M6.csv', names=['y','u'])
A4M1_df = pd.read_csv('MOB_Dataset1_A4_M7.csv', names=['y','u'])
A4M2_df = pd.read_csv('MOB_Dataset1_A4_M8.csv', names=['y','u'])
A5M1_df = pd.read_csv('MOB_Dataset1_A5_M9.csv', names=['y','u'])
A5M2_df = pd.read_csv('MOB_Dataset1_A5_M10.csv', names=['y','u'])
A6M1_df = pd.read_csv('MOB_Dataset1_A6_M11.csv', names=['y','u'])
A6M2_df = pd.read_csv('MOB_Dataset1_A6_M12.csv', names=['y','u'])

athlete_list = [A1M1_df, A1M2_df, A2M1_df, A2M2_df, A3M1_df, A3M2_df, A4M1_df, A4M2_df, A5M1_df, A5M2_df, A6M1_df, A6M2_df] #12 (6x2)
athlete_list_raw = [A1M1_df, A1M2_df, A2M1_df, A2M2_df, A3M1_df, A3M2_df, A4M1_df, A4M2_df, A5M1_df, A5M2_df, A6M1_df, A6M2_df].copy() #12 (6x2)


code_name_a1 = ['A1M1', 'A1M2', 'A2M1', 'A2M2', 'A3M1', 'A3M2', 'A4M1', 'A4M2', 'A5M1', 'A5M2', 'A6M1', 'A6M2'] #12

## Create lists with different scalers

Create list athlete_list_scaled01 for scaling y and u between 0 and 1

In [ ]:
# minmaxscl = MinMaxScaler()
# athlete_list_scale_d01 = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     lst_scaled['u'] = minmaxscl.fit_transform(lst_scaled[['u']])
#     lst_scaled['y'] = minmaxscl.fit_transform(lst_scaled[['y']])
#     athlete_list_scale_d01.append( lst_scaled)


Create list athlete_list_scaled_11 for scaling y and u between -1 and 1

In [ ]:
# minmaxscl = MinMaxScaler((-1,1))
# athlete_list_scaled_11 = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     lst_scaled['u'] = minmaxscl.fit_transform(lst_scaled[['u']])
#     lst_scaled['y'] = minmaxscl.fit_transform(lst_scaled[['y']])
#     athlete_list_scaled_11.append( lst_scaled)

# # for i in range( len(athlete_list_scaled_11)):
# #     lst = athlete_list_scaled_11[i]
# #     print(f'#{i} - {code_name_a1[i]}: shape {lst.shape}, U:[{lst["u"].min()}, {lst["u"].max()}], y:[{lst["y"].min()}, {lst["y"].max()}]')


Create list athlete_list_scaled_std for standard scaling of y and u

In [ ]:
# standardscl = StandardScaler()
# athlete_list_scale_std = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     lst_scaled['u'] = standardscl.fit_transform(lst_scaled[['u']])
#     lst_scaled['y'] = standardscl.fit_transform(lst_scaled[['y']])
#     athlete_list_scale_std.append( lst_scaled)

# # for i in range( len(athlete_list_scale_std)):
# #     lst = athlete_list_scale_std[i]
# #     print(f'#{i} - {code_name_a1[i]}: shape {lst.shape}, U:[{lst["u"].min()}, {lst["u"].max()}], y:[{lst["y"].min()}, {lst["y"].max()}]')


Create list athlete_list_scaled_robust for Robust scaling of y and u

In [ ]:
# robustscl = RobustScaler()
# athlete_list_scale_robust = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     lst_scaled['u'] = robustscl.fit_transform(lst_scaled[['u']])
#     lst_scaled['y'] = robustscl.fit_transform(lst_scaled[['y']])
#     athlete_list_scale_robust.append( lst_scaled)

# # for i in range( len(athlete_list_scale_robust)):
# #     lst = athlete_list_scale_robust[i]
# #     print(f'#{i} - {code_name_a1[i]}: shape {lst.shape}, U:[{lst["u"].min()}, {lst["u"].max()}], y:[{lst["y"].min()}, {lst["y"].max()}]')


Create list athlete_list_subtr_min where all numbers are subtracted of the minimum value


In [ ]:
# athlete_list_scale_subtr_min = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     minvals = lst_scaled.min()
#     lst_scaled = lst_scaled - minvals
#     athlete_list_scale_subtr_min.append( lst_scaled)


Create list athlete_list_subtr_first where all numbers are subtracted of the first value

In [ ]:
# athlete_list_scale_subtr_first = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     firstvals = [lst["y"][0], lst["u"][0]]
#     lst_scaled = lst_scaled - firstvals
#     athlete_list_scale_subtr_first.append( lst_scaled)


Calculate moving average

In [ ]:
# mov_avg_wind = 3
# athlete_list_mov_wind_5 = []
# for lst in athlete_list:
#     lst_scaled = lst.copy()
#     lst_scaled = lst_scaled.rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
#     # lst_scaled["u"] = lst_scaled["u"].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
#     # lst_scaled["y"] = lst_scaled["y"].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
#     athlete_list_mov_wind_5.append( lst_scaled)


## Preprocessing:<br>
Calculate moving average<br>
Then standardize the data


In [ ]:
standardscl = StandardScaler()

mov_avg_wind = 3
for i in range(len(athlete_list)):
    lst = athlete_list[i]
    lst_scaled = lst.copy()
    lst_scaled = lst_scaled.rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
    # lst_scaled["u"] = lst_scaled["u"].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
    # lst_scaled["y"] = lst_scaled["y"].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()

    lst_scaled['u'] = standardscl.fit_transform(lst_scaled[['u']])
    lst_scaled['y'] = standardscl.fit_transform(lst_scaled[['y']])

    athlete_list[i] = lst_scaled


# print(athlete_list[0][0:10])
# print(athlete_list_raw[0][0:10])



#### Plot inputs and outputs, normal and moving average

Create list athlete_list_scaled_std for standard scaling of y and u

In [ ]:
iter_min = 2000
iter_max = 2500

mov_avg_wind = 3


for i, athlete in enumerate(athlete_list): # enumerate to get the index and athlete_dataframe
#for i, athlete in enumerate(athlete_list_raw): # enumerate to get the index and athlete_dataframe
  #Start all values at Zero
  print(f"Athlete:{athlete.shape}")

  #Ploting using two axis with different scales
  fig, ax1 = plt.subplots(figsize=(20, 6))
  ax1.plot(athlete['u'][iter_min:iter_max], color='blue', label='Input')
  ax1.set_xlabel('X-axis Time')
  ax1.set_ylabel('U1-axis Input', color='blue')
  ax1.tick_params('y', colors='blue')
  ax2 = ax1.twinx()
  #trend = mk.original_test(athlete['y'])
  #ax3.plot(trend, color='green', label='Output')
  y_mean = np.mean(athlete['y'])
#   print("Mean of y:", y_mean)
#   print("Variance of y:", np.var(athlete['y']))
  ax2.plot(athlete['y'][iter_min:iter_max], color='red', label='Output')
  ax2.set_ylabel('Y2-axis Output', color='red')
  ax2.tick_params('y', colors='red')
  fig.legend(loc="upper right")
  fig.suptitle(f"Athlete: {code_name_a1[i]}")
  fig.tight_layout()

  ################
  # SUBTRACT MINIMUM

#   if g_normalize_minmax:
#     athlete['u'] = (athlete['u'] - athlete['u'].min()) / (athlete['u'].max() - athlete['u'].min())
#     athlete['y'] = (athlete['y'] - athlete['y'].min()) / (athlete['y'].max() - athlete['y'].min())

#   if g_normalize_stdscal:
#     athlete['u'] = (athlete['u'] - athlete['u'].mean()) / athlete['u'].std()
#     athlete['y'] = (athlete['y'] - athlete['y'].mean()) / athlete['y'].std()


#   mov_avrg_u = athlete['u'][iter_min:iter_max].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
#   mov_avrg_y = athlete['y'][iter_min:iter_max].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()


  ax1.tick_params('y', colors='blue')
  #ax1.plot(mov_avrg_u, color='green', label='Mov-Mean Input')
  ax1.plot(athlete_list[i][["u"]][iter_min:iter_max], color='green', label='Mov-Mean Input')

  #ax2.plot(mov_avrg_y, color='orange', label='Mov-Mean Output')
  ax1.plot(athlete_list[i][["y"]][iter_min:iter_max], color='orange', label='Mov-Mean Output')

  ax2.set_ylabel('Y2-axis Input', color='green')
  ax2.tick_params('y', colors='green')

model_generator(athlete_list_raw[0], 1, 1, 1)



## Pre-Processing && Plot

#### I'm using this cell just for tests

In [ ]:
#1ºData Preparation
#Pre-processing

#High frequency disturbances (too high measurement frequency -> Downsampling)
#1ºlow pass filter | 2º Downsampling


#Outliers
dataset = athlete_list[8]

#dataset['u'] = dataset['u'] - np.mean(dataset['u'])
#dataset['y'] = dataset['y'] - np.mean(dataset['y'])

mov_avrg_u = dataset['u'].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
mov_avrg_y = dataset['y'].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
#result = mk.original_test(data)
trend = mk.original_test(dataset['y'])
#plot
fig, ax1 = plt.subplots(figsize=(20, 6))

#ax1.plot(dataset['y'], color='blue', label='Real_Output')
#ax1.plot(mov_avrg_y, color='red', label='Mov-Mean-output')
ax1.set_xlabel('X-axis Time')
ax1.set_ylabel('U1-axis Output', color='blue')
ax1.tick_params('y', colors='blue')

ax2 = ax1.twinx()
ax2.plot(dataset['u'], color='green', label='Input')
ax2.plot(mov_avrg_u, color='orange', label='Mov-Mean_Input')

ax2.set_ylabel('Y2-axis Input', color='green')
ax2.tick_params('y', colors='green')

fig.legend(loc="upper right")
fig.tight_layout()


#### For each athlete/game, plot inputs (u) and outputs (y)

In [ ]:

for i, athlete in enumerate(athlete_list_raw): # enumerate to get the index and athlete_dataframe
  #Start all values at Zero
  print(f"Athlete:{athlete.shape}")

  #Ploting using two axis with different scales
  fig, ax1 = plt.subplots(figsize=(20, 6))
  ax1.plot(athlete['u'], color='blue', label='Input')
  ax1.set_xlabel('X-axis Time')
  ax1.set_ylabel('U1-axis Input', color='blue')
  ax1.tick_params('y', colors='blue')
  ax2 = ax1.twinx()
  #trend = mk.original_test(athlete['y'])
  #ax3 = ax1.twinx()
  #ax3.plot(trend, color='green', label='Output')
  y_mean = np.mean(athlete['y'])
  print("Mean of y:", y_mean)
  print("Variance of y:", np.var(athlete['y']))
  ax2.plot(athlete['y'], color='red', label='Output')
  ax2.set_ylabel('Y2-axis Output', color='red')
  ax2.tick_params('y', colors='red')
  fig.legend(loc="upper right")
  fig.suptitle(f"Athlete: {code_name_a1[i]}")
  fig.tight_layout()


#### For each athlete and game, plot moving average of inputs (u) and outputs (y)

In [ ]:
#1ºData Preparation
#Pre-processing && Ploting
limit = 0
for i, athlete in enumerate(athlete_list): # enumerate to get the index and athlete_dataframe
  #Start all values at Zero
  print(f"Athlete:{athlete.shape}")
  #athlete['u'] = athlete['u'] - athlete['u'][0] #0.38
  #athlete['y'] = athlete['y'] - athlete['y'][0]

  if g_subtract_min:
    athlete['u'] = athlete['u'] - athlete['u'].min() #0.64
    athlete['y'] = athlete['y'] - athlete['y'].min()

  athlete['u'] = athlete['u'].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()
  athlete['y'] = athlete['y'].rolling(window=mov_avg_wind, center=True, min_periods=1).mean()

#   athlete['u'] = athlete['u'].rolling(window=2, center=True, min_periods=1).mean()
#   athlete['y'] = athlete['y'].rolling(window=2, center=True, min_periods=1).mean()


  #Ploting using two axis with different scales
  fig, ax1 = plt.subplots(figsize=(20, 6))
  ax1.plot(athlete['u'], color='blue', label='Input')
  ax1.set_xlabel('X-axis Time')
  ax1.set_ylabel('U1-axis Input', color='blue')
  ax1.tick_params('y', colors='blue')
  ax2 = ax1.twinx()
  print("athlete['u']",'Max:',athlete['u'].max(),'Min:',athlete['u'].min())
  print("athlete['y']",'Max:',athlete['y'].max(),'Min:',athlete['y'].min())

  #trend = mk.original_test(athlete['y'])
  #ax3 = ax1.twinx()
  #ax3.plot(trend, color='green', label='Output')

  '''  if athlete.shape[0] == 7743 and limit == 0:
    print("tEAWDAWDrue")
    athlete_list[0]['u'] = athlete['u']
    athlete_list[0]['y'] = athlete['y']
    limit +=+ 1'''
  #y_mean = np.mean(athlete['y'])
  #print("Mean of y:", y_mean)
  #print("Variance of y:", np.var(y))
  ax2.plot(athlete['y'], color='red', label='Output')
  ax2.set_ylabel('Y2-axis Output', color='red')
  ax2.tick_params('y', colors='red')
  fig.legend(loc="upper right")
  fig.suptitle(f"Athlete: {code_name_a1[i]}")
  fig.tight_layout()


## 2º System Identification

#### Test models based on na_max, nb_max and xdelay_max, then store results in results_df dataframe

In [ ]:
from re import X
# Find the best model structure (time-invariant modelling)
results_df = pd.DataFrame(columns=['Athlete_Code','Model_Index', 'na', 'nb', 'd', 'AIC', 'Rsq','RMSE','BIC','FPE'])
selected_models = pd.DataFrame(columns=['Athlete_Code','ParamCount','ParamValues','Metrics'])  #ParamCount[na:nb:d] || ParamValues[na,nb] || Metrics[Rsq, AIC, FPE]

# Calculate ARX models to the evaluation data with all possible parameters to find the best fit by checking r-squared and AIC
na_max = nb_max = xdelay_max = 2
model_count=0

##Loop through parameters
#Starts 1:1:0 .... 3,3,3               (1,na_max|1,nb_max|xdelay_max) (max=4)
#Starts 1:1:0 .... 4,4,4               (1,na_max+1|1,nb_max+1|xdelay_max+1)(max=4)
#Starts 0:1:0 .... 3,4,4               (na_max|1,nb_max+1|xdelay_max+1)(max=4)

for i in range(len(athlete_list)):
  data = athlete_list[i]
  model_count=0
  print(f"Athlete: {code_name_a1[i]}")
  for na in range(1,na_max+1):
    for nb in range(1,nb_max+1):
      for d in range(xdelay_max+1):

        print('\nModel #{} Dataset:{} \nwith {} a parameters, {} b parameters and {} time delay'.format(model_count,code_name_a1[i],na,nb,d))
        #load the associated data using dataloader(data, na, nb, d)
        X_loader, y_loader = dataloader(data, na,  nb, d)
        #print("X_loader:",X_loader)#print("y_loader:",y_loader)


        #Correction to use python## Only when values are equal to zero
        if X_loader.empty:#
            print("Skipping model due to empty X_loader.")
            continue # proceed to next iteration
        if y_loader.empty:
            print("Skipping model due to empty y_loader.")
            continue # proceed to next iteration


        ##Calculate the a and b parameters
        y_loader, ysim, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, na, nb, d)


        ## Model evaluation for each generated model use r_squared and aic def above
        print("its regression yield r squared: {}, Akaike's information criteria: {}".format(r_squared, aic))



        # Save relevant information to the empty lists
        new_row = pd.DataFrame({'Athlete_Code': code_name_a1[i],'Model_Index': [model_count], 'na': [na], 'nb': [nb], 'd': [d], 'AIC': [aic], 'Rsq': [r_squared],'RMSE':[Rmse], 'BIC':[bic],'FPE':[fpe], 'Parameters_a':[parameters_a], 'Parameters_b':[parameters_b]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)
        # Update the model counter
        model_count += 1


  #Plot for each player/game

  fig, ax1 = plt.subplots(figsize=(20, 6))
  X_axis = np.arange(len(results_df)) #results_df['Model_Index']

  #For R-square Error
  ax1.plot(X_axis, results_df['Rsq'], color='blue', label='R-squared')
  ax1.set_xlabel('Model Index', fontsize=14)  # Increased font size
  ax1.set_ylabel('R-squared', color='blue', fontsize=14)  # Increased font size
  ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)  # Increased tick label size

  #For AIC
  ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
  ax2.plot(X_axis, results_df['AIC'], color='red', label='AIC')
  ax2.set_ylabel('AIC', color='red', fontsize=14)  # Increased font size
  ax2.tick_params(axis='y', labelcolor='red', labelsize=12)  # Increased tick label size

  #For RMSE
  ax3 = ax1.twinx()
  ax3.plot(X_axis, results_df['RMSE'], color='green', label='RMSE', linestyle='--')
  ax3.set_ylabel('RMSE', color='green', fontsize=14)
  ax3.tick_params(axis='y', labelcolor='green', labelsize=12)

  #For FPE
  ax4 = ax1.twinx()
  ax4.plot(X_axis, results_df['FPE'], color='purple', label='FPE', linestyle=':')
  ax4.set_ylabel('RMSE', color='purple', fontsize=14)
  ax4.tick_params(axis='y', labelcolor='purple', labelsize=12)

  #For BIC
  ax5 = ax4.twinx()
  ax5.plot(X_axis, results_df['BIC'], color='orange', label='BIC', linestyle='-.')
  ax5.set_ylabel('BIC', color='orange', fontsize=14)
  ax5.tick_params(axis='y', labelcolor='orange', labelsize=12)

  # General graphic configs
  fig.legend(loc="upper right", fontsize=12)  # Add a legend for both plots, increased font size
  ax1.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
  ax2.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
  plt.title('Model Evaluation Metrics vs. Model Index', fontsize=16)  # Added title with increased font size
  plt.tight_layout()  # Adjust layout to prevent overlapping elements
plt.show()

In [ ]:
X_loader

In [ ]:
results_df

In [ ]:
#Select Best Models
scaler = MinMaxScaler()
normalized_metrics = scaler.fit_transform(results_df[['AIC', 'Rsq', 'FPE', 'BIC', 'RMSE']])
# Create a new DataFrame with the normalized metrics
normalized_df = pd.DataFrame(normalized_metrics, columns=['AIC_norm', 'Rsq_norm', 'FPE_norm', 'BIC_norm', 'RMSE_norm'])
selection_df = pd.concat([results_df, normalized_df], axis=1)
na_max = nb_max = xdelay_max = 2

#best_models_df = pd.DataFrame(columns=['Athlete_Code','ParamCount','ParamValues','Metrics'])  #ParamCount[na:nb:d] || ParamValues[na,nb] || Metrics[Rsq, AIC, FPE]
best_models_df = pd.DataFrame()


num_models = na_max*nb_max*(xdelay_max+1)
cases = len(results_df)/(num_models) #12

#LOOP for each case (models)
for i in range(int(cases)):
  print("Case: ",i,"Start: ",(i*num_models),"End: ",(i+1)*num_models)
  #Select each dataset models data
  selected_df = selection_df[(i*num_models):(i+1)*num_models].copy()

  #Create a Ranking Selection
  selected_df['AIC_rank'] = selected_df['AIC'].rank()
  selected_df['BIC_rank'] = selected_df['BIC'].rank()

  # Rank by Rsq
  selected_df['Rsq_rank'] = selected_df['Rsq'].rank(ascending=False)  # Note: ascending=False for Rsq

  # Calculate the combined ranking
  selected_df['Ranking'] = selected_df['Rsq_rank'] + selected_df['AIC_rank']#+ selected_df['BIC_rank']

  best_df = selected_df.sort_values(by='Ranking', ascending=True).iloc[0]
  #new_row = pd.DataFrame({'Athlete_Code':[filter['Athlete_Code'][0]],'ParamCount'["{}:{}:{}".format(filter['na'][0],filter['nb'][0],filter['d'][0])],'ParamValues':[],'Metrics':[filter['Rsq'][0],filter['AIC'][0],filter['FPE'][0]]})

  #print(best_df)
  #print(selected_df.sort_values(by='Ranking', ascending=True))



  new_row = pd.DataFrame({'Athlete_Code': [best_df['Athlete_Code']],
                          'na': [best_df['na']],
                          'nb': [best_df['nb']],
                          'd': [best_df['d']],
                          'Ranking': [best_df['Ranking']],
                          'AIC_rank': [best_df['AIC_rank']],
                          'Rsq_rank': [best_df['Rsq_rank']],
                          'BIC_rank': [best_df['BIC_rank']],
                          'Rsq': [best_df['Rsq']],
                          'AIC': [best_df['AIC']],
                          'BIC': [best_df['BIC']],
                          'FPE': [best_df['FPE']],
                          'Parameters_a':[best_df['Parameters_a']],
                          'Parameters_b':[best_df['Parameters_b']]})

  best_models_df = pd.concat([best_models_df, new_row], ignore_index=True)

  #print(selected_df.sort_values(by='Rsq', ascending=False)[:3])
  #print(selected_df.sort_values(by='AIC', ascending=True)[:3])
  #print(selected_df.sort_values(by='FPE', ascending=True)[:3])


In [ ]:
#results_df.sort_values(by='Rsq', ascending=False)

best_models_df["Rsq"].mean()

In [ ]:
# from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=best_models_df)


In [ ]:
#Simulate Best Models
r2_total = 0
for i in range(len(best_models_df)):
  print(f"Athlete: {code_name_a1[i]}")
  #Define Individual Values
  data = athlete_list[i]
  number_a = best_models_df['na'].iloc[i]
  number_b = best_models_df['nb'].iloc[i]
  d = best_models_df['d'].iloc[i]



  y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, number_a, number_b, d)

  #Ploting
  plt.figure(figsize=(20, 6))
  X_axis = np.arange(0,y_loader.shape[0])
  #print(X_axis)

  plt.plot(data['y'], label='Real Value (y_loader)', color='blue')
  #plt.plot(data['u'], label='Real Input (u_loader)', color='purple', linestyle=':')
  # Plot the original data (y)    *AUTOMATIC
  plt.plot(X_axis, y_auto, label='Predicted (y_auto)', color='red', linestyle='--')

  # Add labels and title
  plt.xlabel('Time / Index')
  plt.ylabel('Values')
  plt.title(f'{code_name_a1[i]}     [{number_a},{number_b},{d}]\nR-squared: {r_squared:.3f}, AIC: {aic:.1f}')
  plt.legend()  # Add a legend
  plt.grid(True) # Add grid lines for better readability
  # Adjust the y-axis limits if necessary for better visualization
  #plt.ylim([50, 200])
  plt.show()
  print(y_auto[:10])


  r2_total += r_squared
print(f"Average Total R^2: {r2_total/12}")

## Simulate ONE model for ALL Samples

In [ ]:
#Simulation ONE model structure
one_4_all = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','SSG','TC' ,'AIC', 'Rsq','RMSE','BIC','FPE'])
number_a=1
number_b=1
d=1

rsq_sum = 0
model_count=0
for i in range(len(athlete_list)):
  data = athlete_list[i]
  print(f"Athlete: {code_name_a1[i]}")

  #Model Generator
  y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, number_a, number_b, d)
  print("regression yield r squared: {}, Akaike's information criteria: {}".format(r_squared, aic))

  #Save Results to "one_4_all"
  new_row = pd.DataFrame({'Athlete_Code': code_name_a1[i],'SSG':[SSG],'TC':[tau], 'Parameters_a':[parameters_a], 'Parameters_b':[parameters_b], 'na': [number_a], 'nb': [number_b], 'd': [d], 'AIC': [aic], 'Rsq': [r_squared],'RMSE':[Rmse], 'BIC':[bic],'FPE':[fpe]})
  one_4_all = pd.concat([one_4_all, new_row], ignore_index=True)


  model_count += 1
  rsq_sum += r_squared


  ##Ploting
  #
  plt.figure(figsize=(20, 6))
  X_axis = np.arange(0,y_loader.shape[0])
  print(X_axis)
  # Plot the actual values (y_loader)
  plt.plot(data['y'], label='Real Value', color='blue')
  # plt.plot(data['u'], label='Real Input (u_loader)', color='purple', linestyle=':')
  # Plot the original data (y)    *AUTOMATIC
  plt.plot(X_axis, y_auto, label='Predicted Value', color='red', linestyle='--')

  # Add labels and title
  plt.xlabel('Time / Index')
  plt.ylabel('Values')
  plt.title(f'Comparison: Real VS Prediction\nR-squared: {r_squared:.3f}, AIC: {aic:.1f}')
  plt.legend()  # Add a legend
  plt.grid(True) # Add grid lines for better readability
  # Adjust the y-axis limits if necessary for better visualization
  #plt.ylim([50, 200])

  plt.show()

print(f"Average Total R^2 Error: {rsq_sum/12}")
#Do the parameter estimates vary a lot between the subjects or between the volleyball matches?


In [ ]:
one_4_all

In [ ]:
#Do the parameter estimates vary a lot between the subjects or between the volleyball matches?
athlete_codes = one_4_all['Athlete_Code'].to_list()

#Plot for each player/game
fig, ax1 = plt.subplots(figsize=(20, 6))
X_axis = athlete_codes #results_df['Model_Index']

#plot parameters A
ax1.plot(X_axis, one_4_all['Parameters_a'].str[0].tolist(), color='blue', label='A1')
ax1.set_xlabel('Athlete Code', fontsize=14)  # Increased font size
ax1.set_ylabel('Value', fontsize=14)  # Increased font size
ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)  # Increased tick label size

#Plot parameters B
ax1.plot(X_axis, one_4_all['Parameters_b'].str[0].tolist(), color='red', label='B0')
#ax1.plot(X_axis, one_4_all['Parameters_b'].str[1].tolist(), color='orange', label='B1')

'''#First param B
x2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(X_axis, one_4_all['Param_b'].str[0].tolist(), color='red', label='AIC')
#ax2.plot(X_axis, one_4_all['Param_b'].str[1].tolist(), color='red', label='AIC')
ax2.set_ylabel('Param_b', color='red', fontsize=14)  # Increased font size
ax2.tick_params(axis='y', labelcolor='red', labelsize=12)  # Increased tick label size
'''


# General graphic configs
fig.legend(loc="upper right", fontsize=12)  # Add a legend for both plots, increased font size
ax1.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
ax2.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
plt.title('Model Parameters for each dataset', fontsize=16)  # Added title with increased font size
plt.tight_layout()  # Adjust layout to prevent overlapping elements
plt.show()


In [ ]:
one_4_all["Rsq"].mean()


#### Basically the same as the previous chart, but just for parameter b

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract the 'b' parameter values and athlete codes
b_values = one_4_all['Parameters_b'].to_list()
athlete_codes = one_4_all['Athlete_Code'].to_list()

num_athletes = len(athlete_codes)
num_params = len(b_values[0])  # Assuming all athletes have the same number of 'b' parameters

# Create an array to store the parameter values for each position
param_positions = [[] for _ in range(num_params)]
for athlete_params in b_values:
    for i, param_value in enumerate(athlete_params):
        param_positions[i].append(param_value)

# Create the plot
plt.figure(figsize=(10, 6))  # Adjust figure size as needed

for i, position_values in enumerate(param_positions):
    plt.plot(athlete_codes, position_values, label=f"Position {i}")

# Customize the plot
plt.xlabel("Athlete Code")
plt.ylabel("Parameter Value")
plt.title("B parameter comparison by position")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability

plt.show()

In [ ]:
best_models_df

In [ ]:
one_4_all

In [ ]:
# Store best_models_df and one_for_all in the dictionary with R2 results

g_r2_dictionary["INVAR_BestModels"] = best_models_df[["Athlete_Code", "na", "nb", "d", "Rsq"]]
g_r2_dictionary["INVAR_OneForAll"] = one_4_all[["Athlete_Code", "na", "nb", "d", "Rsq"]]


In [ ]:
# Biological interpretation

#First Order Model
## has 1 a-parameters
#  Code Here
##


#Second Order Model
## has 2 a-parameters
#  Code Here
##




#Athlete Average Moder   (???)
#structure: 1:2:0
#for p_case in best_models_df.iterrows():
counter = 0
number_a=1
number_b=2
d=0
athlete_mean_model = pd.DataFrame(columns=['Athlete', 'Parameters_a', 'Parameters_b', 'Rsq_M1', 'AIC_M1', 'Rsq_M2', 'AIC_M2'])

for pcase in one_4_all.iterrows():
  #print(pcase)

  if pcase[1]["Athlete_Code"][3] == '1':    #First Game
    params_a_memory = pcase[1]['Parameters_a']
    params_b_memory = pcase[1]['Parameters_b']
    counter += 1

  else: #second game
    mean_a = (params_a_memory + pcase[1]['Parameters_a']) / 2
    mean_b = (params_b_memory + pcase[1]['Parameters_b']) / 2

    #Simulation
    #Simulation for first Game
    y_loader, ysim, parameters_a, parameters_b, SSG, tau, r_squared1, aic1, Rmse1, bic1, fpe1 = model_generator(athlete_list[counter-1], number_a, number_b, d)

    ## Simulate values of Y
    #Plot Simulation
    print("Game1:",r_squared1)
    plt.plot(y_loader, label='Real Value (y_loader)', color='blue')
    plt.plot(np.arange(len(ysim)), ysim, label='Predicted Mean (ysim)', color='red', linestyle='--')
    plt.xlabel('Time / Index')
    plt.ylabel('Values')
    plt.title('Athlete:{}  Game: 1 '.format(pcase[1]['Athlete_Code'][:2]))
    plt.legend()  # Add a legend
    plt.grid(True) # Add grid lines for better readability
    plt.show()


      ##Simulation for Second Game
    X_loader, y_loader = dataloader(athlete_list[counter], number_a, number_b, d)
    y_loader, ysim, parameters_a, parameters_b, SSG, tau, r_squared2, aic2, Rmse2, bic2, fpe2 = model_generator(athlete_list[counter], number_a, number_b, d)

    #Plot Simulation
    print("Game2:",r_squared2)
    plt.plot(y_loader, label='Real Value (y_loader)', color='green')
    plt.plot(np.arange(len(ysim)), ysim, label='Predicted Mean (ysim)', color='orange', linestyle='--')
    plt.xlabel('Time / Index')
    plt.ylabel('Values')
    plt.title('Athlete:{}  Game: 2 '.format(pcase[1]['Athlete_Code'][:2]))
    plt.legend()  # Add a legend
    plt.grid(True) # Add grid lines for better readability
    plt.show()


    #Save to the Dataframe
    new_row = pd.DataFrame({'Athlete': [pcase[1]['Athlete_Code'][:2]],'Parameters_a':[mean_a], 'Parameters_b':[mean_b], 'Rsq_M1':[r_squared1], 'AIC_M1':[aic1], 'Rsq_M2':[r_squared2], 'AIC_M2':[aic2]})
    athlete_mean_model = pd.concat([athlete_mean_model, new_row], ignore_index=True)
    plt.show()







In [ ]:
athlete_mean_model

 ## Time-varying dynamics

In the lines bellow we consider that the parameter could change in time. This allows the moddels to have different values,depending on the time, allowing the model to better explain/predict the system behavior.


#### Used to generate synthetic data

In [ ]:
# na = 1
# nb = 1
# d = 1

# #INCORPORATE ASSIGNMENT DATA
# X_loader, y_loader = dataloader(athlete_list[1], na, nb, d)
# N = len(y_loader)
# y = y_loader#.values.reshape(-1,1)
# u = X_loader['u-{}'.format(d)].values.reshape(-1,1)

# N = len(y_loader)
# t = np.arange(N)                                        # example for 'time'/points 0-(N-1)
# f = 2 * np.pi / 40                                      # f = chosen frequence for square wave


# e = np.random.randn(N, 1) * np.sqrt(0.16)               # Noise vector e-N(0,0.16), 'standard normal distribution'is mean 0, standard deviation 1.
# a1 = np.ones((N,1)) * -1                                # a1 = constant
# a2 = np.ones((N,1)) * 0.25                              # a2 = constant (if you have a2 means it is second order system)
# b0 = np.ones((N,1))
# b0[200:899] = 2                                         # b0 = variable
# # delay = 1


# y = np.zeros((N,1))                                     # start with an empty vector for y
# for k in range(max(na, nb+d-1), N):
#     y[k] = -a1[k] * y[k-1] - a2[k] * y[k-2] + b0[k] * u[k-d] + e[k]  # if we have a2 means it is a second order system.

# plt.plot(u)
# plt.plot(y)
# plt.show()


### 1. Time-variant 1 - Recursive ARX, RLS (Recursive Least Squares) algorithm

In [ ]:
def fRLS(chart_title = "Time-variant - Recursive ARX, RLS (Recursive Least Squares) algorithm", na=2, nb=1, d=1, showCharts=False, showPrints=False):
  #Time-variant - recursive ARX

    if na != 2 or nb != 1 or d != 1:
        raise ValueError("Invalid values for 'na', 'nb' or 'd'. 'na' must be 2, 'nb' must be 1 and 'd' must be 1.")


    df2_rsquared = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','Rsq'])

    r2_total = 0

    for i in range(len(athlete_list)):
        data = athlete_list[i]

        #INCORPORATE ASSIGNMENT DATA
        X_loader, y_loader = dataloader(athlete_list[i], na, nb, d)
        N = len(y_loader)

        y = y_loader#.values.reshape(-1,1)
        u = X_loader['u-{}'.format(d)].values.reshape(-1,1)

        #print("N:", N)
        #print("y:", y.shape)
        #print("u:", u.shape)

        t = np.arange(N)                                        # example for 'time'/points 0-(N-1)
        f = 2 * np.pi / 40                                      # f = chosen frequence for square wave


        #dm  =3                              # Number of parameters to estimate
        dm  = na + nb                              # Number of parameters to estimate

        # Initialise matrix
        X = np.zeros((dm,1))              # prepare an input matrix that can be used in the loop, size is the number of parameters
        P = 1000 * np.eye(dm)            # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
        a_hat = np.ones((dm,1))         # Minimum value to not create error in case of 0 division


        # Information to be stored in the loop
        a1est = np.zeros((N,1))
        a2est = np.zeros((N,1))
        b0est = np.zeros((N,1))


        # Loop over all the data
        for k in range(max(na, nb+d-1), N):
            # 1. Update the matrix X with the new data
            X[0] = - y[k-1]
            X[1] = - y[k-2]
            X[2] = u[k-d]

            # 2. Update gain matrix g
            g = (P @ X) / (1 +( X.T @ P @ X))

            # 3. Update covariance matrix P
            P = P - g @ X.T @ P

            # 4. Re-estimate the parameter coefficients
            yhat =  X.T @ a_hat      # y = data vector * parameters vetor
            error = y[k] - yhat
            a_hat = a_hat + g * error
            # OR: a_hat = a_hat + g * ( y(k) - (X' * a_hat))

            # 5. Store estimates (get the diagonal elements of a_hat that correspond to a1, a2, b0)
            a1est[k] = a_hat[0]
            a2est[k] = a_hat[1]
            b0est[k] = a_hat[2]

        # Simulate model (step 2)
        ysim2 = np.zeros((N,1))
        for k in range(max(na, nb+d-1), N):
            ysim2[k] = - a1est[k] * ysim2[k-1] - a2est[k] * ysim2[k-2] + b0est[k] * u[k-d]

        r_squared = r2_score(y, ysim2)
        new_row = pd.DataFrame([[code_name_a1[i], na, nb, d, r_squared]], columns=df2_rsquared.columns)
        df2_rsquared = pd.concat([df2_rsquared, new_row], ignore_index=True)

        r2_total += r_squared

        if showCharts:

            plt.title(f"Param estim - {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}")
            plt.plot(a1est, label="A1 est", color="blue")
            plt.plot(a2est, label="A2 est", color="cyan")
            plt.plot(b0est, label="B1 est", color="green")
            plt.xlabel('Time / Index')
            plt.ylabel('Values')
            plt.legend()
            plt.tight_layout()
            plt.show()


            plt.title(f"Real vs predic, {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}. R2: {r2_score(y, ysim2):.3f}")
            plt.plot(y, label="y real", color="blue")
            plt.plot(ysim2, label="y pred", color="red")
            plt.xlabel('Time / Index')
            plt.ylabel('Values')
            plt.legend()
            plt.tight_layout()
            plt.show()


        if showPrints:

            print('R2:',r2_score(y, ysim2))

    if showPrints:
        print("Average R2: {r2_total/12}")


    return df2_rsquared


In [ ]:
dfRLS = fRLS(showCharts=False, showPrints=False)
g_r2_dictionary["RLS"] = dfRLS


### 2. Time-variant 2 - Recursive ARX RDW - Rectangular Data Weighing

In [ ]:
def fRDW(win_size = 200, chart_title = "Time-variant - Recursive ARX RDW (Rectangular Data Weighing) algorithm", na=2, nb=1, d=1, showCharts=False, showPrints=False):
  #Time-variant - recursive ARX

  if na != 2 or nb != 1 or d != 1:
    raise ValueError("Invalid values for 'na', 'nb' or 'd'. 'na' must be 2, 'nb' must be 1 and 'd' must be 1.")

  # Time-variant - recursive ARX

  chart_title = f"{chart_title} (win={win_size})"
  #df = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','SSG','TC' ,'AIC', 'Rsq','RMSE','BIC','FPE'])
  df3_rsquared = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','Rsq'])
  r2_total = 0


  for i in range(len(athlete_list)):
    data = athlete_list[i]

    #INCORPORATE ASSIGNMENT DATA
    X_loader, y_loader = dataloader(athlete_list[i], na, nb, d)
    N = len(y_loader)

    y = y_loader#.values.reshape(-1,1)
    u = X_loader['u-{}'.format(d)].values.reshape(-1,1)

    t = np.arange(N)                                        # example for 'time'/points 0-(N-1)
    f = 2 * np.pi / 40                                      # f = chosen frequence for square wave

    #dm  =3                              # Number of parameters to estimate
    dm  = na + nb                              # Number of parameters to estimate

    # Initialise matrix
    X = np.zeros((dm,1))              # prepare an input matrix that can be used in the loop, size is the number of parameters
    P = 1000 * np.eye(dm)            # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
    a_hat = np.ones((dm,1))         # Minimum value to not create error in case of 0 division


    # Initialise matrix
    X_add = np.zeros((dm,1))
    X_remove = np.zeros((dm,1))
    P = 1000 * np.eye(dm)             # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
    a_hat = np.ones((dm,1))           # Minimum value to don't create error in case of 0 division


    # Information to be stored in the loop
    a1est = np.zeros((N,1))
    a2est = np.zeros((N,1))
    b0est = np.zeros((N,1))



    for k in range(max(na, nb+d-1), N):
        # 1. Update the matrix X with the new data
        X_add[0] = -y[k-1]
        X_add[1] = -y[k-2]
        X_add[2] = u[k-1]

        # 2. Update gain matrix
        g = P @ X_add / (1 + X_add.T @ P @ X_add)  # inverse of matrix is 'devision'

        # 3. Update covariance matrix P
        P = P - g @ X_add.T @ P

        # 4. Re-estimate the parameter coefficients based on the new data
        yhat = X_add.T @ a_hat
        error = y[k] - yhat
        a_hat = a_hat + g * error

        # 5 Remove effect of old data every data at (k-s)th instant
        if k - win_size - max(nb + d - 1, na) > 0:
            # 5.a Update the matrix X(k-s) with the new data
            X_remove[0] = -y[k - win_size - 1] #a1
            X_remove[1] = -y[k - win_size - 2] #a2
            X_remove[2] = u[k - win_size - d] #b0
            # 5.b Update gain matrix
            g = -P @ X_remove / (1 + X_remove.T @ P @ X_remove)
            # 5.c Update covariance matrix P
            P = (P - g @ X_remove.T @ P)
            # 5.d Re-estimate the parameter coefficients based on the new data
            yhat = X_remove.T @ a_hat
            error = y[k - win_size] - yhat
            a_hat = a_hat + g * error

        a1est[k] = a_hat[0]
        a2est[k] = a_hat[1]
        b0est[k] = a_hat[2]


    # Simulate model (step 3)
    ysim3 = np.zeros((N,1))
    for k in range(max(na, nb+d-1), N):
        ysim3[k] = -a1est[k] * ysim3[k-1] - a2est[k] * ysim3[k-2] + b0est[k] * u[k-d]



    r_squared = r2_score(y, ysim3)
    #df.append(code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe)
    #df.add_row([code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe])
    new_row = pd.DataFrame([[code_name_a1[i], na, nb, d, r_squared]], columns=df3_rsquared.columns)
    df3_rsquared = pd.concat([df3_rsquared, new_row], ignore_index=True)

    r2_total += r_squared


    if showCharts:
        plt.title(f"Param estim - {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}")
        plt.plot(a1est, label="A1 est", color="blue")
        plt.plot(a2est, label="A2 est", color="cyan")
        plt.plot(b0est, label="B1 est", color="green")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()

        #   fig, ax1 = plt.subplots(figsize=(20, 6))
        #   fig.suptitle(f"Athlete: {code_name_a1[i]}")
        #   fig.tight_layout()


        plt.title(f"Real vs predic, {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}. R2: {r2_score(y, ysim3):.3f}")
        plt.plot(y, label="y real", color="blue")
        plt.plot(ysim3, label="y pred", color="red")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()


        if showPrints:

            print('R2:',r2_score(y, ysim3))
            # print('R2:',r_2(y, ysim2))

    if showPrints:
        print("Average R2: {r2_total/12}")




  return df3_rsquared


In [ ]:
def fRDW_121(win_size = 200, chart_title = "Time-variant - Recursive ARX RDW (Rectangular Data Weighing) algorithm", na=1, nb=2, d=1, showCharts=False, showPrints=False):
  #Time-variant - recursive ARX

  if na != 1 or nb != 2 or d != 1:
    raise ValueError("Invalid values for 'na', 'nb' or 'd'. 'na' must be 1, 'nb' must be 2 and 'd' must be 1.")

  # Time-variant - recursive ARX

  chart_title = f"{chart_title} (win={win_size})"
  #df = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','SSG','TC' ,'AIC', 'Rsq','RMSE','BIC','FPE'])
  df3_rsquared = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','Rsq'])
  r2_total = 0


  for i in range(len(athlete_list)):
    data = athlete_list[i]

    #INCORPORATE ASSIGNMENT DATA
    X_loader, y_loader = dataloader(athlete_list[i], na, nb, d)
    N = len(y_loader)

    y = y_loader#.values.reshape(-1,1)
    u = X_loader['u-{}'.format(d)].values.reshape(-1,1)

    t = np.arange(N)                                        # example for 'time'/points 0-(N-1)
    f = 2 * np.pi / 40                                      # f = chosen frequence for square wave

    #dm  =3                              # Number of parameters to estimate
    dm  = na + nb                              # Number of parameters to estimate

    # Initialise matrix
    X = np.zeros((dm,1))              # prepare an input matrix that can be used in the loop, size is the number of parameters
    P = 1000 * np.eye(dm)            # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
    a_hat = np.ones((dm,1))         # Minimum value to not create error in case of 0 division


    # Initialise matrix
    X_add = np.zeros((dm,1))
    X_remove = np.zeros((dm,1))
    P = 1000 * np.eye(dm)             # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
    a_hat = np.ones((dm,1))           # Minimum value to don't create error in case of 0 division


    # Information to be stored in the loop
    a1est = np.zeros((N,1))
    b0est = np.zeros((N,1))
    b1est = np.zeros((N,1))



    for k in range(max(na, nb+d-1), N):
        # 1. Update the matrix X with the new data
        # X_add[0] = -y[k-1]
        # X_add[1] = u[k-1]
        # X_add[2] = u[k-2]
        X_add[0] = -y[k-1]
        X_add[1] = u[k-1]
        X_add[2] = u[k-2]

        # 2. Update gain matrix
        g = P @ X_add / (1 + X_add.T @ P @ X_add)  # inverse of matrix is 'devision'

        # 3. Update covariance matrix P
        P = P - g @ X_add.T @ P

        # 4. Re-estimate the parameter coefficients based on the new data
        yhat = X_add.T @ a_hat
        error = y[k] - yhat
        a_hat = a_hat + g * error

        # 5 Remove effect of old data every data at (k-s)th instant
        if k - win_size - max(nb + d - 1, na) > 0:
            # 5.a Update the matrix X(k-s) with the new data
            X_remove[0] = -y[k - win_size - 1] #a1
            X_remove[1] = -y[k - win_size - 2] #a2
            X_remove[2] = u[k - win_size - d] #b0
            # 5.b Update gain matrix
            g = -P @ X_remove / (1 + X_remove.T @ P @ X_remove)
            # 5.c Update covariance matrix P
            P = (P - g @ X_remove.T @ P)
            # 5.d Re-estimate the parameter coefficients based on the new data
            yhat = X_remove.T @ a_hat
            error = y[k - win_size] - yhat
            a_hat = a_hat + g * error

        a1est[k] = a_hat[0]
        b0est[k] = a_hat[1]
        b1est[k] = a_hat[2]


    # Simulate model (step 3)
    ysim3 = np.zeros((N,1))
    for k in range(max(na, nb+d-1), N):
        ysim3[k] = -a1est[k] * ysim3[k-1] + b0est[k] * u[k-d-1] + b1est[k] * u[k-d]



    r_squared = r2_score(y, ysim3)
    #df.append(code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe)
    #df.add_row([code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe])
    new_row = pd.DataFrame([[code_name_a1[i], na, nb, d, r_squared]], columns=df3_rsquared.columns)
    df3_rsquared = pd.concat([df3_rsquared, new_row], ignore_index=True)

    r2_total += r_squared


    if showCharts:
        plt.title(f"Param estim - {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}")
        plt.plot(a1est, label="A1 est", color="blue")
        plt.plot(b0est, label="B0 est", color="cyan")
        plt.plot(b1est, label="B1 est", color="green")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()

        #   fig, ax1 = plt.subplots(figsize=(20, 6))
        #   fig.suptitle(f"Athlete: {code_name_a1[i]}")
        #   fig.tight_layout()


        plt.title(f"Real vs predic, {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}. R2: {r2_score(y, ysim3):.3f}")
        plt.plot(y, label="y real", color="blue")
        plt.plot(ysim3, label="y pred", color="red")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()


        if showPrints:

            print('R2:',r2_score(y, ysim3))
            # print('R2:',r_2(y, ysim2))

    if showPrints:
        print("Average R2: {r2_total/12}")




  return df3_rsquared


In [ ]:
for win in [50, 100, 150, 200]:
    dfRDW = fRDW_121(win_size = win, showCharts=False, showPrints=False)
    g_r2_dictionary["RDW121_{}".format(win)] = dfRDW


### 3. Time-variant 3 - Recursive ARX EWP - Exponential Weighted Past

In [ ]:
def fEWP(Alpha = 0.985, chart_title = "Time-variant - Recursive ARX EWP (Exponential Weighting Past) algorithm", na=2, nb=1, d=1, showCharts=False, showPrints=False):
  #Time-variant - recursive ARX

  if na != 2 or nb != 1 or d != 1:
    raise ValueError("Invalid values for 'na', 'nb' or 'd'. 'na' must be 2, 'nb' must be 1 and 'd' must be 1.")

  # Time-variant - recursive ARX

  chart_title = f"{chart_title} (alpha={Alpha})"

  #Time-variant - recursive ARX with EWP (exponentially weighted past)


  df4_rsquared = pd.DataFrame(columns=['Athlete_Code', 'na', 'nb', 'd','Rsq'])
  r2_total = 0

  for i in range(len(athlete_list)):
    data = athlete_list[i]


    #INCORPORATE ASSIGNMENT DATA
    X_loader, y_loader = dataloader(athlete_list[i], na, nb, d)
    N = len(y_loader)

    y = y_loader#.values.reshape(-1,1)
    u = X_loader['u-{}'.format(d)].values.reshape(-1,1)

    t = np.arange(N)                                        # example for 'time'/points 0-(N-1)
    f = 2 * np.pi / 40                                      # f = chosen frequence for square wave

    #dm  =3                              # Number of parameters to estimate
    dm  = na + nb                              # Number of parameters to estimate

    # Initialise matrix
    X = np.zeros((dm,1))              # prepare an input matrix that can be used in the loop, size is the number of parameters
    P = 1000 * np.eye(dm)            # Covariance matrix set to large value because of uncertainty at the beginning due to lack of measurement
    a_hat = np.ones((dm,1))         # Minimum value to not create error in case of 0 division


    # Information to be stored in the loop
    a1est = np.zeros((N,1))
    a2est = np.zeros((N,1))
    b0est = np.zeros((N,1))


    #Loop over all the data
    for k in range(max(na, nb+d-1), N):

        # 1. Update the matrix X with the new data
        X[0] = -y[k-1]
        X[1] = -y[k-2]
        X[2] = u[k-d]

        # 2. Update gain matrix
        g = P @ X / (Alpha + X.T @ P @ X)

        # 3. Update covariance matrix P
        P = (P - g @ X.T @ P) / Alpha

        # 4. Re-estimate the parameter coefficients based on the new data
        yhat = X.T @ a_hat
        error = y[k] - yhat
        a_hat = a_hat + g * error

        a1est[k] = a_hat[0]
        a2est[k] = a_hat[1]
        b0est[k] = a_hat[2]


    # Simulate model (step 3)
    ysim4 = np.zeros((N,1))
    for k in range(max(na, nb+d-1), N):
        ysim4[k] = -a1est[k] * ysim4[k-1] - a2est[k] * ysim4[k-2] + b0est[k] * u[k-d]



    r_squared = r2_score(y, ysim4)
    #df.append(code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe)
    #df.add_row([code_name_a1[i], na, nb, d, SSG, tau, aic, r_squared, Rmse, bic, fpe])
    new_row = pd.DataFrame([[code_name_a1[i], na, nb, d, r_squared]], columns=df4_rsquared.columns)
    df4_rsquared = pd.concat([df4_rsquared, new_row], ignore_index=True)

    r2_total += r_squared

    if showCharts:

        plt.title(f"Param estim - {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}")
        plt.plot(a1est, label="A1 est", color="blue")
        plt.plot(a2est, label="A2 est", color="cyan")
        plt.plot(b0est, label="B1 est", color="green")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()


        plt.title(f"Real vs predic, {chart_title}\nAthlete {code_name_a1[i][:2]}, Game {code_name_a1[i][-1]}. R2: {r2_score(y, ysim4):.3f}")
        plt.plot(y, label="y real", color="blue")
        plt.plot(ysim4, label="y pred", color="red")
        plt.xlabel('Time / Index')
        plt.ylabel('Values')
        plt.legend()
        plt.tight_layout()
        plt.show()





    if showPrints:

        print('R2:',r2_score(y, ysim4))
        # print('R2:',r_2(y, ysim2))

  if showPrints:
    print("Average R2: {r2_total/12}")



  return df4_rsquared



In [ ]:

for alp in [.95, .98, .99, .995, .999]:
    dfEWP = fEWP(Alpha = alp, showCharts=False, showPrints=False)
    g_r2_dictionary["EWP_{}".format(alp)] = dfEWP


### Combine results

In [ ]:
dfcomb = g_r2_dictionary[list(g_r2_dictionary.keys())[0]].drop(["Rsq"], axis=1)
for key in g_r2_dictionary.keys():
    dfcomb[key] = g_r2_dictionary[key]["Rsq"]
meanrow = {"Athlete_Code": "Mean", "na":"", "nb":"", "d":""}
for key in g_r2_dictionary.keys():
    meanrow[key] = dfcomb[key].mean()


dfcomb.loc[len(dfcomb)] = meanrow

dfcomb


In [ ]:
dfcomb

In [ ]:
#2ºSystem Identification
def calcmodel(data, na, nb, d):
    # Load the associated data using dataloader(data, na, nb, d)

    X_loader, y_loader = dataloader(data, na, nb, d)


    # Estimate the a and b parameters vector [a0...an, b0...bm].T by the formula inv(X.T dot X) dot X.T dot y
    params = np.linalg.inv(X_loader.T @ X_loader) @ X_loader.T @ y_loader
    #params = np.linalg.solve(X_loader.T, X_loader) @ X_loader.T @ y_loader
    params = params.values


    parameters_a = params[:na]
    parameters_b = params[na:]


    # Optional: Calculate the predicted values of Y via Matrix-Vector multiplication (this is for prediction, not simulation)
    Y_pred = np.dot(X_loader, params)


    #Calculate a simulation using the predicted parameters use the following structure:
    ysim = np.zeros((len(X_loader)+1,1))  # the plus one is because the first value is initialized and not simulated

    ysim[0] = y_loader.values[0]    # initialize the first value
    max_lag = max(na, nb + d)  # Maximum lag across both endogenous and exogenous variables

    for k in range(max_lag, len(X_loader)):
        #Add the contribution from autoregressive terms (endogenous variables)
        for i_a in range(1, na + 1):
            #ysim[k] += parameters_a[i_a - 1] * ysim[k - i_a]
            ysim[k] += parameters_a[i_a - 1] * X_loader.iloc[k, i_a]

        #Add the contribution from exogenous input ter\ms (with delay d)
        for j_b in range(1, nb + 1):
            #ysim[k] += estimation[i_a - 1] * ysim[k - i_a - X_loader['u-{}'.format(d)]]                    #to get the column at delay u-d use this syntax X_loader['u-{}'.format(d)]
            ysim[k] += parameters_b[j_b-1] * X_loader['u-{}'.format(d)].iloc[k-j_b]


    # Calculate r-squared
    y_loader = y_loader.values.reshape(-1,1)
    SS_res = np.sum((y_loader - ysim[1:]) ** 2)  # Residual sum of squares
    SS_tot = np.sum((y_loader - np.mean(y_loader)) ** 2)  # Total sum of squares
    rsq = 1 - (SS_res / SS_tot)
    #print("ss_red:",SS_res,"ss_tot",SS_tot,"RSQ:",rsq)


    # Calculate AIC
    n = len(y_loader)  # Number of observations
    # Number of parameters (including intercept)
    aic =  2 * (na + nb) + len(y_loader) * np.log(1 - rsq) # Evaluates the complexicity of the Model


    return Y_pred, ysim, params, rsq, aic




#print(calcmodel(data,0,1,0))
#print(calcmodel(data,1,0,0))


# 2º Practical assignment

In [ ]:
uploaded = files.upload()

In [ ]:
M_334_df = pd.DataFrame()
M_343_df = pd.DataFrame()
WT_df = pd.DataFrame()
Unknown_df = pd.DataFrame()

''' [ 4X types
  [Wild], [Mut1], [Mut2], [unkown]
  [C1, C2, C3, C4, C5, C6]# Wildtype
  [[y,u](C1), [y,u](C2), [y,u](C3), [y,u], [y,u], [y,u]]
  []
  []

]

'''


# Mutant 334
for i in range(1,7):
  load_input = pd.read_csv('Mutation334_C{}_In.csv'.format(i), names=['u_C{}'.format(i)])
  load_output = pd.read_csv('Mutation334_C{}_Out.csv'.format(i), names=['y_C{}'.format(i)])
  M_334_df = pd.concat([M_334_df, load_input, load_output], axis=1)

# Mutant 343
for i in range(1,7):
  load_input = pd.read_csv('Mutation343_C{}_In.csv'.format(i), names=['u_C{}'.format(i)])
  load_output = pd.read_csv('Mutation343_C{}_Out.csv'.format(i), names=['y_C{}'.format(i)])
  M_343_df = pd.concat([M_343_df, load_input, load_output], axis=1)

# WildType
for i in range(1,7):
  load_input = pd.read_csv('WildType_C{}_In.csv'.format(i), names=['u_C{}'.format(i)])
  load_output = pd.read_csv('WildType_C{}_Out.csv'.format(i), names=['y_C{}'.format(i)])
  WT_df = pd.concat([WT_df, load_input, load_output], axis=1)


# WildType
for i in range(1,3):
  load_input = pd.read_csv('Unknown_C{}_In.csv'.format(i), names=['u_C{}'.format(i)])
  load_output = pd.read_csv('Unknown_C{}_Out.csv'.format(i), names=['y_C{}'.format(i)])
  Unknown_df = pd.concat([Unknown_df, load_input, load_output], axis=1)

In [ ]:

# Mutant 334

M334_C1_In_df = pd.read_csv('Mutation334_C1_In.csv', names=['u'])
M334_C2_In_df = pd.read_csv('Mutation334_C2_In.csv', names=['u'])
M334_C3_In_df = pd.read_csv('Mutation334_C3_In.csv', names=['u'])
M334_C4_In_df = pd.read_csv('Mutation334_C4_In.csv', names=['u'])
M334_C5_In_df = pd.read_csv('Mutation334_C5_In.csv', names=['u'])
M334_C6_In_df = pd.read_csv('Mutation334_C6_In.csv', names=['u'])

M334_C1_Out_df = pd.read_csv('Mutation334_C1_Out.csv', names=['y'])
M334_C2_Out_df = pd.read_csv('Mutation334_C2_Out.csv', names=['y'])
M334_C3_Out_df = pd.read_csv('Mutation334_C3_Out.csv', names=['y'])
M334_C4_Out_df = pd.read_csv('Mutation334_C4_Out.csv', names=['y'])
M334_C5_Out_df = pd.read_csv('Mutation334_C5_Out.csv', names=['y'])
M334_C6_Out_df = pd.read_csv('Mutation334_C6_Out.csv', names=['y'])

Mutant_334_list = [M334_C1_In_df,
M334_C2_In_df,
M334_C3_In_df,
M334_C4_In_df,
M334_C5_In_df,
M334_C6_In_df,
M334_C1_Out_df,
M334_C2_Out_df,
M334_C3_Out_df,
M334_C4_Out_df,
M334_C5_Out_df,
M334_C6_Out_df] #12

# Mutant 343

M343_C1_In_df = pd.read_csv('Mutation343_C1_In.csv', names=['u'])
M343_C2_In_df = pd.read_csv('Mutation343_C2_In.csv', names=['u'])
M343_C3_In_df = pd.read_csv('Mutation343_C3_In.csv', names=['u'])
M343_C4_In_df = pd.read_csv('Mutation343_C4_In.csv', names=['u'])
M343_C5_In_df = pd.read_csv('Mutation343_C5_In.csv', names=['u'])
M343_C6_In_df = pd.read_csv('Mutation343_C6_In.csv', names=['u'])

M343_C1_Out_df = pd.read_csv('Mutation343_C1_Out.csv', names=['y'])
M343_C2_Out_df = pd.read_csv('Mutation343_C2_Out.csv', names=['y'])
M343_C3_Out_df = pd.read_csv('Mutation343_C3_Out.csv', names=['y'])
M343_C4_Out_df = pd.read_csv('Mutation343_C4_Out.csv', names=['y'])
M343_C5_Out_df = pd.read_csv('Mutation343_C5_Out.csv', names=['y'])
M343_C6_Out_df = pd.read_csv('Mutation343_C6_Out.csv', names=['y'])

Mutant_343_list = [M343_C1_In_df,
M343_C2_In_df,
M343_C3_In_df,
M343_C4_In_df,
M343_C5_In_df,
M343_C6_In_df,
M343_C1_Out_df,
M343_C2_Out_df,
M343_C3_Out_df,
M343_C4_Out_df,
M343_C5_Out_df,
M343_C6_Out_df] #12

# WildType

WildType_C1_In_df = pd.read_csv('WildType_C1_In.csv', names=['u'])
WildType_C2_In_df = pd.read_csv('WildType_C2_In.csv', names=['u'])
WildType_C3_In_df = pd.read_csv('WildType_C3_In.csv', names=['u'])
WildType_C4_In_df = pd.read_csv('WildType_C4_In.csv', names=['u'])
WildType_C5_In_df = pd.read_csv('WildType_C5_In.csv', names=['u'])
WildType_C6_In_df = pd.read_csv('WildType_C6_In.csv', names=['u'])

WildType_C1_Out_df = pd.read_csv('WildType_C1_Out.csv', names=['y'])
WildType_C2_Out_df = pd.read_csv('WildType_C2_Out.csv', names=['y'])
WildType_C3_Out_df = pd.read_csv('WildType_C3_Out.csv', names=['y'])
WildType_C4_Out_df = pd.read_csv('WildType_C4_Out.csv', names=['y'])
WildType_C5_Out_df = pd.read_csv('WildType_C5_Out.csv', names=['y'])
WildType_C6_Out_df = pd.read_csv('WildType_C6_Out.csv', names=['y'])

WildType_list = [WildType_C1_In_df,
WildType_C2_In_df,
WildType_C3_In_df,
WildType_C4_In_df,
WildType_C5_In_df,
WildType_C6_In_df,
WildType_C1_Out_df,
WildType_C2_Out_df,
WildType_C3_Out_df,
WildType_C4_Out_df,
WildType_C5_Out_df,
WildType_C6_Out_df] #12

# Unknown

Unknown_C1_In_df = pd.read_csv('Unknown_C1_In.csv', names=['u'])
Unknown_C2_In_df = pd.read_csv('Unknown_C2_In.csv', names=['u'])

Unknown_C1_Out_df = pd.read_csv('Unknown_C1_Out.csv', names=['y'])
Unknown_C2_Out_df = pd.read_csv('Unknown_C2_Out.csv', names=['y'])

UnknownType_list = [Unknown_C1_In_df,
Unknown_C2_In_df,
Unknown_C1_Out_df,
Unknown_C2_Out_df] #4


In [ ]:
M_343_df[118:125]

In [ ]:
for variation in [M_334_df, M_343_df, WT_df, Unknown_df]:
  for i in range(0,len(variation.columns),2):
    input = variation.columns[i]
    output = variation.columns[i+1]
    #print(input, output)
    data = pd.concat([variation[input], variation[output]], axis=1)
    data2 = variation[[input, output]].copy()
    if (data.values == data2.values).all():
      continue
      #print('Inputs are all the Equal!')

data2

In [ ]:
#Note: All inputs(u) are Equal

for variation in [M_334_df, M_343_df, WT_df, Unknown_df]:
  counter = 1
  #input = variation.iloc[119:, 0] # Access column by index using iloc
  #plt.plot(input, label='Input'.format(counter))
  input = variation.iloc[100:, 0] # Access column by index using iloc
  plt.plot(input, label='Input'.format(counter))
  for i in range(0,len(variation.columns),2): #0,2,..8,10 (6X)
    #variation.iloc[:, i] = variation.iloc[:, i] - variation.iloc[:, i].min()
    #variation.iloc[:, i] = variation.iloc[:, i] - variation.iloc[:, i].mean()
    #variation.iloc[:, i] = variation.iloc[:, i] / variation.iloc[:, i].std()

    output = variation.iloc[119:, i+1] # Access next column for output



    plt.plot(output, label='Output C{}'.format(counter))
    plt.legend()
    counter += 1
  plt.show()


code_name = ["M_334","M_343","WT","Unknown"]



In [ ]:
#1ºData Preparation
#Pre-processing && Ploting
for lists in [Mutant_334_list,Mutant_343_list,WildType_list,UnknownType_list]:
  pds = pd.DataFrame(columns=['Input','Output'])


#Remove the first 119 values (0 or 1) for all the datasets and reset the index for 0
#all_datasets_df = [df.iloc[120:].reset_index(drop=True) for df in [M_334_df, M_343_df, WT_df, Unknown_df]]

#all_datasets_df = [[M_334_df, M_343_df, WT_df, Unknown_df]]    #Original Datasets


In [ ]:
#Set values < 0 to eq 0

all_datasets_df = [M_334_df, M_343_df, WT_df, Unknown_df]
count = 0
for variation in all_datasets_df:
  for i in range(0,len(variation.columns),2):
    input = variation.columns[i]
    output = variation.columns[i+1]

    for iter in range(len(variation[output])):
      value = variation[output][iter]
      if value < 0:
        #variation.loc[iter, output] = 0
        print(all_datasets_df[count][output][iter])
  count += 1


#all_datasets_df = [df.iloc[100:].reset_index(drop=True) for df in all_datasets_df]

# 2. System identification

In [ ]:
all_datasets_df = [df.iloc[119:] for df in [M_334_df, M_343_df, WT_df, Unknown_df]]
all_datasets_df[3]

In [ ]:
na_max = nb_max = xdelay_max = 5
model_count=0

all_datasets_df = [M_334_df, M_343_df, WT_df, Unknown_df]
final_results_df = pd.DataFrame(columns=['Code_Name','Model_Index','na','nb','d','AIC','Rsq','RMSE','BIC','FPE','Parameters_a','Parameters_b'])
r2_sum = 0
##Loop through parameters
#Starts 1:1:0 .... 4,4,4               (1,na_max+1|1,nb_max+1|xdelay_max+1)(max=4)
#Starts 0:1:0 .... 3,4,4               (na_max|1,nb_max+1|xdelay_max+1)(max=4)
counter = nm_count = 0
for variation in all_datasets_df:
  print("Running: ",code_name[nm_count])
  for i in range(0,len(variation.columns),2):
    data = pd.DataFrame(columns=['Input','Output'])
    data['u'] = variation[variation.columns[i]]
    data['y'] = variation[variation.columns[i+1]]
    print(variation.columns[i],variation.columns[i+1])

    #input = variation.columns[i]
    #output = variation.columns[i+1]
    #data = pd.concat([variation[input].rename("u"), variation[output].rename("y")], axis=1)
    model_counter = 0

    for na in range(1,na_max+1):
      for nb in range(1,nb_max+1):
        for d in range(xdelay_max+1):
          #print(data)
          #Calculate Model
          y_loader, ysim, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, na, nb, d)

          #Save Results
          new_row = pd.DataFrame({'Code_Name': code_name[nm_count],'Model_Index': [model_counter], 'na': [na], 'nb': [nb], 'd': [d], 'AIC': [aic], 'Rsq': [r_squared],'RMSE':[Rmse], 'BIC':[bic],'FPE':[fpe], 'Parameters_a':[parameters_a], 'Parameters_b':[parameters_b]})
          final_results_df = pd.concat([final_results_df, new_row], ignore_index=True)
          r2_sum += r_squared

          model_counter += 1
  nm_count += 1

print("Average R2:", r2_sum/(len(all_datasets_df)*na_max*nb_max*xdelay_max))




In [ ]:
#Plot for each player/game
fig, ax1 = plt.subplots(figsize=(20, 6))
X_axis = np.arange(len(final_results_df)) #results_df['Model_Index']

#For R-square Error
ax1.plot(X_axis, final_results_df['Rsq'], color='blue', label='R-squared')
ax1.set_xlabel('Model Index', fontsize=14)  # Increased font size
ax1.set_ylabel('R-squared', color='blue', fontsize=14)  # Increased font size
ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)  # Increased tick label size

#For AIC
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(X_axis, final_results_df['AIC'], color='red', label='AIC')
ax2.set_ylabel('AIC', color='red', fontsize=14)  # Increased font size
ax2.tick_params(axis='y', labelcolor='red', labelsize=12)  # Increased tick label size

#For RMSE
ax3 = ax1.twinx()
ax3.plot(X_axis, final_results_df['RMSE'], color='green', label='RMSE', linestyle='--')
ax3.set_ylabel('RMSE', color='green', fontsize=14)
ax3.tick_params(axis='y', labelcolor='green', labelsize=12)

#For FPE
ax4 = ax1.twinx()
ax4.plot(X_axis, final_results_df['FPE'], color='purple', label='FPE', linestyle=':')
ax4.set_ylabel('RMSE', color='purple', fontsize=14)
ax4.tick_params(axis='y', labelcolor='purple', labelsize=12)

#For BIC
ax5 = ax4.twinx()
ax5.plot(X_axis, final_results_df['BIC'], color='orange', label='BIC', linestyle='-.')
ax5.set_ylabel('BIC', color='orange', fontsize=14)
ax5.tick_params(axis='y', labelcolor='orange', labelsize=12)

# General graphic configs
fig.legend(loc="upper right", fontsize=12)  # Add a legend for both plots, increased font size
ax1.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
ax2.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
plt.title('Model Evaluation Metrics vs. Model Index', fontsize=16)  # Added title with increased font size
plt.tight_layout()  # Adjust layout to prevent overlapping elements
plt.show()

In [ ]:
interval = 5*5*6
for value in range(int(len(final_results_df)/(interval))): #0..19 (20x)
  #Plot for each player/game

  selection = final_results_df[(value*interval):(value+1)*interval]

  fig, ax1 = plt.subplots(figsize=(20, 6))
  X_axis = np.arange(len(selection)) #results_df['Model_Index']

  #For R-square Error
  ax1.plot(X_axis, selection['Rsq'], color='blue', label='R-squared')
  ax1.set_xlabel('Model Index', fontsize=14)  # Increased font size
  ax1.set_ylabel('R-squared', color='blue', fontsize=14)  # Increased font size
  ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)  # Increased tick label size

  #For AIC
  ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
  ax2.plot(X_axis, selection['AIC'], color='red', label='AIC')
  ax2.set_ylabel('AIC', color='red', fontsize=14)  # Increased font size
  ax2.tick_params(axis='y', labelcolor='red', labelsize=12)  # Increased tick label size

  #For RMSE
  ax3 = ax1.twinx()
  ax3.plot(X_axis, selection['RMSE'], color='green', label='RMSE', linestyle='--')
  ax3.set_ylabel('RMSE', color='green', fontsize=14)
  ax3.tick_params(axis='y', labelcolor='green', labelsize=12)

  #For FPE
  ax4 = ax1.twinx()
  ax4.plot(X_axis, selection['FPE'], color='purple', label='FPE', linestyle=':')
  ax4.set_ylabel('RMSE', color='purple', fontsize=14)
  ax4.tick_params(axis='y', labelcolor='purple', labelsize=12)

  #For BIC
  ax5 = ax4.twinx()
  ax5.plot(X_axis, selection['BIC'], color='orange', label='BIC', linestyle='-.')
  ax5.set_ylabel('BIC', color='orange', fontsize=14)
  ax5.tick_params(axis='y', labelcolor='orange', labelsize=12)

  # General graphic configs
  fig.legend(loc="upper right", fontsize=12)  # Add a legend for both plots, increased font size
  ax1.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
  ax2.grid(True, linestyle='--', alpha=0.7)  # Added grid with dashed lines and transparency
  plt.title(f'Model Evaluation Metrics vs. Model Index               Sample:{value}', fontsize=16)  # Added title with increased font size
  plt.tight_layout()  # Adjust layout to prevent overlapping elements
  plt.show()

In [ ]:
final_results_df

In [ ]:
final_results_df.sort_values(by='Rsq', ascending=False)[:50]

In [ ]:
#Select Best Models
scaler = MinMaxScaler()
normalized_metrics = scaler.fit_transform(final_results_df[['AIC', 'Rsq', 'FPE', 'BIC', 'RMSE']])
# Create a new DataFrame with the normalized metrics
normalized_df = pd.DataFrame(normalized_metrics, columns=['AIC_norm', 'Rsq_norm', 'FPE_norm', 'BIC_norm', 'RMSE_norm'])
selection_df = pd.concat([final_results_df, normalized_df], axis=1)


best_models_df = pd.DataFrame()


num_models = na_max*nb_max*(xdelay_max+1) # 5X5X6
cases = len(final_results_df)/(num_models) #3000 / 150 = 20

#LOOP for each case (models)
for i in range(int(cases)): #0,1..19
  print("Case: ",i,"Start: ",(i*num_models),"End: ",(i+1)*num_models)
  #Select each dataset models data
  selected_df = selection_df[(i*num_models):(i+1)*num_models].copy()

  #Create a Ranking Selection
  selected_df['AIC_rank'] = selected_df['AIC'].rank()
  selected_df['BIC_rank'] = selected_df['BIC'].rank()

  # Rank by Rsq
  selected_df['Rsq_rank'] = selected_df['Rsq'].rank(ascending=True)  # Note: ascending=False for Rsq

  # Calculate the combined ranking
  selected_df['Ranking'] = selected_df['Rsq_rank'] + selected_df['AIC_rank']#+ selected_df['BIC_rank']
  best_df = selected_df.sort_values(by='Ranking', ascending=True).iloc[0]

  new_row = pd.DataFrame({'Code_Name': [best_df['Code_Name']],
                          'na': [best_df['na']],
                          'nb': [best_df['nb']],
                          'd': [best_df['d']],
                          'Ranking': [best_df['Ranking']],
                          'AIC_rank': [best_df['AIC_rank']],
                          'Rsq_rank': [best_df['Rsq_rank']],
                          'BIC_rank': [best_df['BIC_rank']],
                          'Rsq': [best_df['Rsq']],
                          'AIC': [best_df['AIC']],
                          'BIC': [best_df['BIC']],
                          'FPE': [best_df['FPE']],
                          'Parameters_a':[best_df['Parameters_a']],
                          'Parameters_b':[best_df['Parameters_b']]})

  #Save new model
  best_models_df = pd.concat([best_models_df, new_row], ignore_index=True)

  #print(selected_df.sort_values(by='Rsq', ascending=False)[:3])
  #print(selected_df.sort_values(by='AIC', ascending=True)[:3])
  #print(selected_df.sort_values(by='FPE', ascending=True)[:3])

In [ ]:
best_models_df

In [ ]:
for x in all_datasets_df:
  #print(x.columns[(i)*2:(i*2)+1])
  print(x.columns)
  for y in range(0,len(x.columns),2):
    sample = pd.DataFrame(columns=['y','u'])
    sample['y'] = x[x.columns[y]]
    sample['u'] = x[x.columns[y+1]]
    #print(x.columns[y],x.columns[y+1
    print(x.columns[y],x.columns[y+1])
    print(y)
    y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(sample, 2, 2, 1)


In [ ]:
best_models_df#.sort_values(by='Ranking', ascending=True)

In [ ]:
#Simulate Best Models
r2_total = 0
for i in range(len(best_models_df)):
  print(f"Case: {[best_models_df['Code_Name'][i]]}_{i%6}")
  #Define Individual Values
  data = all_datasets_df[i]

  print(data.columns)
  for dataset in range(0,len(data.columns),2):
    print(data.columns[dataset],data.columns[dataset+1])


    sample = pd.DataFrame(columns=['y','u'])
    number_a = best_models_df['na'].iloc[i]
    number_b = best_models_df['nb'].iloc[i]
    d = best_models_df['d'].iloc[i]
    sample['y'] = data[data.columns[dataset+1]]
    sample['u'] = data[data.columns[dataset]]

    print(dataset)

    y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(sample, number_a, number_b, d)
    data = sample
    #Ploting
    plt.figure(figsize=(20, 6))
    X_axis = np.arange(0,y_loader.shape[0])
    #print(X_axis)

    plt.plot(data['y'], label='Real Value (y_loader)', color='blue')
    plt.plot(data['u'], label='Real Input (u_loader)', color='purple', linestyle=':')
    # Plot the original data (y)    *AUTOMATIC
    plt.plot(X_axis, y_auto, label='Predicted Automatic (y_auto)', color='red', linestyle='--')

    # Add labels and title
    plt.xlabel('Time / Index')
    plt.ylabel('Values')
    plt.title(f'{code_name[i]}     [{number_a},{number_b},{d}]\nR-squared: {r_squared:.3f}, AIC: {aic:.1f}')
    plt.legend()  # Add a legend
    plt.grid(True) # Add grid lines for better readability
    # Adjust the y-axis limits if necessary for better visualization
    #plt.ylim([50, 200])
    plt.show()
    print(y_auto[:10])


    r2_total += r_squared
print(f"Average Total R^2 Error: {r2_total/12}")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_ccf
#Simulation best model structure
number_a=3
number_b=1
d=0
lags_v = 250

input = [M_334_df, M_343_df, WT_df, Unknown_df][0][['u_C1']].rename(columns={'u_C1': "u"}) #Select input column as DataFrame and rename
output = [M_334_df, M_343_df, WT_df, Unknown_df][0][['y_C1']].rename(columns={'y_C1': "y"}) #Select output column as DataFrame and rename
data = pd.concat([input,output], axis=1) #Concatenate along columns (axis=1)


# print the relative dataframe and check its columns
y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, number_a, number_b, d)

# Calculate the noise term (residuals)
noise = y_loader - y_auto  # Assuming 'y' is the actual output column in your 'data' DataFrame
print(noise)
# Autocorrelation of the noise
fig, ax = plt.subplots(figsize=(12, 4))
plot_acf(noise, lags=lags_v, ax=ax)  # Adjust 'lags' as needed
plt.title('Autocorrelation of Noise')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

# Partial Autocorrelation of the noise
fig, ax = plt.subplots(figsize=(12, 4))
plot_pacf(noise, lags=lags_v, ax=ax)  # Adjust 'lags' as needed
plt.title('Partial Autocorrelation of Noise')
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.show()

# Cross-correlation between input (u) and noise
fig, ax = plt.subplots(figsize=(12, 4))
plot_ccf(data['u'].values, noise, lags=lags_v, ax=ax)  # Adjust 'lags' as needed
plt.title('Cross-correlation between Input and Noise')
plt.xlabel('Lag')
plt.ylabel('Cross-correlation')
plt.show()






#Ploting
plt.figure(figsize=(20, 6))
X_axis = np.arange(0,y_loader.shape[0])
print(X_axis)
# Plot the actual values (y_loader)
plt.plot(data['y'], label='Real Value (y_loader)', color='blue')
plt.plot(data['u'], label='Real Input (y_loader)', color='purple', linestyle=':')
# Plot the original data (y)    *AUTOMATIC
plt.plot(X_axis, y_auto, label='Predicted Automatic (y_auto)', color='red', linestyle='--')

# Add labels and title
plt.xlabel('Time / Index')
plt.ylabel('Values')
plt.title(f'Name:{code_name[0]} Comparison of Real data (y_loader) and Model Prediction (y_auto) - R-squared: {r_squared:.4f}\nParameter a:{parameters_a} Parameter b:{parameters_b} ')
plt.legend()  # Add a legend
plt.grid(True) # Add grid lines for better readability
# Adjust the y-axis limits if necessary for better visualization
#plt.ylim([50, 200])
plt.show()

In [ ]:
#Simulation All best model structures
number_a = 3
number_b = 1
d = 0

count = 0
for dataset in all_datasets_df:
  for i in range(0,len(dataset.columns),2):
    name_in = dataset.columns[i]
    name_out = dataset.columns[i+1]

    input = dataset[[name_in]].rename(columns={f'{name_in}': "u"}) #Select input column as DataFrame and rename
    output = dataset[[name_out]].rename(columns={name_out: "y"}) #Select output column as DataFrame and rename
    data = pd.concat([input, output], axis=1) #Concatenate along columns (axis=1)


    # print the relative dataframe and check its columns
    y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, number_a, number_b, d)

    print(ctrl.TransferFunction(parameters_a,parameters_b))

    #Ploting
    plt.figure(figsize=(20, 6))
    X_axis = np.arange(0,y_loader.shape[0])
    # Plot the actual values (y_loader)
    plt.plot(data['y'], label='Real Value (y_loader)', color='blue')
    plt.plot(data['u'], label='Real Input (y_loader)', color='purple', linestyle=':')
    # Plot the original data (y)    *AUTOMATIC
    plt.plot(X_axis, y_auto, label='Predicted Automatic (y_auto)', color='red', linestyle='--')

    # Add labels and title
    plt.xlabel('Time / Index')
    plt.ylabel('Values')
    plt.title(f'Dataset: {code_name[count]}_{i}       R2: {r_squared:.4f}\nParameter a:{parameters_a} Parameter b:{parameters_b} ') # Include R-squared in the title
    plt.legend()  # Add a legend
    plt.grid(True) # Add grid lines for better readability
    # Adjust the y-axis limits if necessary for better visualization
    #plt.ylim([50, 200])
    plt.show()
  count += 1
#Do the parameter estimates vary a lot between the subjects or between the volleyball matches?



In [ ]:
#Simulation best model structure
number_a=5
number_b=5
d=5

data = pd.concat([M_334_df['u_C6'].rename("u"), M_334_df['y_C6'].rename("y")], axis=1)
# print the relative dataframe and check its columns
y_loader, y_auto, parameters_a, parameters_b, SSG, tau, r_squared, aic, Rmse, bic, fpe = model_generator(data, number_a, number_b, d)


#Ploting
plt.figure(figsize=(20, 6))
X_axis = np.arange(0,y_loader.shape[0])
print(X_axis)
# Plot the actual values (y_loader)
plt.plot(data['y'], label='Real Value (y_loader)', color='blue')
plt.plot(data['u'], label='Real Input (y_loader)', color='purple', linestyle=':')
# Plot the original data (y)    *AUTOMATIC
plt.plot(X_axis, y_auto, label='Predicted Automatic (y_auto)', color='red', linestyle='--')

# Add labels and title
plt.xlabel('Time / Index')
plt.ylabel('Values')
plt.title('Comparison of Real data (y_loader) and Model Prediction (y_auto)')
plt.legend()  # Add a legend
plt.grid(True) # Add grid lines for better readability
# Adjust the y-axis limits if necessary for better visualization
#plt.ylim([50, 200])
plt.show()

# Decompose ARX Modell
Every higher order system can be described as a combination of several first order systems.
* Serial coupling
* Parallel coupling
* Feedback coupling



How to find model:
1. Reducing the model order by polynomial division (only when m>n)
2. Decomposition into first order models


In [ ]:
result

In [ ]:
import control as ctrl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def draw_tf_scheme(tf_type, tf1, tf2, tf3):
    """Draws a scheme of the transfer functions using networkx,
    including TF3 in parallel, with nodes in a linear layout."""

    graph = nx.DiGraph()
    graph.add_node("Input", layer=0)
    graph.add_node("Output", layer=2)  # Output layer is now 2

    # Add TF3 in parallel
    graph.add_node("TF3", layer=1)
    graph.add_edges_from([("Input", "TF3"), ("TF3", "Output")])

    if tf_type == "Series":
        graph.add_node("TF1", layer=0)
        graph.add_node("TF2", layer=1)
        graph.add_edges_from([("Input", "TF1"), ("TF1", "TF2"), ("TF2", "Output")])
        pos = nx.multipartite_layout(graph, subset_key="layer")
        plt.title(f"Series Configuration: ({tf1} -> {tf2}) || {tf3}")

    elif tf_type == "Parallel":
        graph.add_node("TF1", layer=1)
        graph.add_node("TF2", layer=1)
        graph.add_edges_from([("Input", "TF1"), ("Input", "TF2"), ("TF1", "Output"), ("TF2", "Output")])
        pos = nx.multipartite_layout(graph, subset_key="layer")
        plt.title(f"Parallel Configuration: ({tf1} || {tf2}) || {tf3}")

    elif tf_type == "Feedback":
        graph.add_node("TF1", layer=1)
        graph.add_node("TF2", layer=1)
        graph.add_edges_from([("Input", "TF1"), ("TF1", "Output"), ("Output", "TF2"), ("TF2", "TF1")])
        pos = nx.multipartite_layout(graph, subset_key="layer")
        plt.title(f"Feedback Configuration: ({tf1} <-> {tf2}) || {tf3}")

    # Draw the graph
    nx.draw(graph, with_labels=True, node_color="skyblue", node_size=1500, pos=pos)
    plt.show()

In [ ]:

def decomposition_model(params_a, params_b): #params_a = den = n  ;   params_b = num = m
  result = {}
  m = len(params_b)
  n = len(params_a)-1
  #print(params_a,params_b)
  #coefficients of the polynomials in decreasing powers
  par_a = np.poly1d(np.flip(params_a))
  par_b = np.poly1d(np.flip(params_b))
  #print(par_a,par_b)

  #Step 1  -  Polynomial Division
  if m >= n:
    quotient, remainder = np.polydiv(par_b, par_a)
    #quotient,remainder = deconvolve(par_b,par_a)
    print(f'Remainder:{remainder}  Quotient:{quotient}')
    params_c = remainder
  else:
    params_c = par_b
    quotient = []
  #Q and C are in parallel
  #print("C:", params_c, "C_0=", params_c[0])
  #print(f'Order a:{len(params_a)}  b:{len(params_a)} c:{len(params_c)} Q:{len(quotient)}\n\n')

  #Step 2 - Decomposition
  #2.a - All possible configurations:
  decompositions = {}
  A1 = params_a[-2]
  A2 = params_a[-1]
  B0 = params_c[0]
  B1 = params_c[1]
  print(f'B0:{B0} B1:{B1} A1:{A1} A2:{A2}')


  #Configurations
  #Series
  s_b1, s_b2, s_a1, s_a2 = serial(B0,B1,A1,A2)
  #Parallel
  p_b1, p_b2, p_a1, p_a2 = parallel(B0,B1,A1,A2)
  #Feedback
  #fr_b1, fr_b2, fr_a1, fr_a2 = reverse_set3(B0,B1,A1,A2)
  fr_b1, fr_b2, fr_a1, fr_a2 = feedback_m(B0,B1,A1,A2)
  f_b1, f_b2, f_a1, f_a2 = feedback(B0,B1,A1,A2)
  fr2_b1, fr2_b2, fr2_a1, fr2_a2 = reverse_set31(B0,B1,A1,A2)
  configurations = { 'Series': {'b1': s_b1, 'b2': s_b2, 'a1': s_a1, 'a2': s_a2},
                    'Parallel': {'b1': p_b1, 'b2': p_b2, 'a1': p_a1, 'a2': p_a2},
                    'Feedback': {'b1': f_b1, 'b2': f_b2, 'a1': f_a1, 'a2': f_a2}}
  #2.b - Remove Imaginary Values
  #print("Configuration Inicial:", configurations)

  #Step 3 - Remove unstable configurations
  #if |a| > 1 ---> 1-a < 0 ---> TC < 0 (not real meaning)
  for types in configurations: #'Series', 'Parallel', 'Feedback'
    unstable_configurations = False
    #print("Checking unstable configuration from: {}",types)

    for param in configurations[types]: #'b1','b2','a1','a2'
      if param.startswith('a'): #or configurations[types][param] == 'a2':
        #check if "a" > 1 and save the configuration if not
        if abs(configurations[types][param]) > 1 or isinstance(configurations[types][param], complex): #second part is for complex numbers
          unstable_configurations = True #mark configuration as "impossible"

    if unstable_configurations:
      configurations[types] = {}
      #result.update({f"{r_types}" : {'TF_1':TF_1,'TF_2':TF_2,'TC_1':TC_1,'TC_2':TC_2, 'G1':K1, 'G2':K2}})
    print(f'Final result: {configurations}\n')


  #Probable real configurations
  for r_types in configurations:
    if configurations[r_types] == {}:
      result.update({f"{r_types}" : {'Unstable'}})
      continue
    #Initialize var
    pre_TF_1 = []
    pre_TF_2 = []
    TF_r = None
    #define the TFs
    for param in configurations[r_types]:
      if param.endswith('1'):
        pre_TF_1.append(configurations[r_types][param]) #(b1,a1)
      elif param.endswith('2'):
        pre_TF_2.append(configurations[r_types][param]) #(b2, a2)
      else:
        print("Unkown situation for:",param,"in",r_types)
    pre_TF_1.append(1)
    pre_TF_2.append(1)
    TF_1 = ctrl.TransferFunction(pre_TF_1[0], pre_TF_1[1:][::-1])
    TF_2 = ctrl.TransferFunction(pre_TF_2[0], pre_TF_2[1:][::-1])
    print(f"Une: {r_types}\n TF1: {TF_1} TF2: {TF_2}")


    #Step 4
    #1 - Time Constants and Gain
    poles1 = ctrl.poles(TF_1)
    TC_1 = -1 / poles1
    K1 = ctrl.dcgain(TF_1) # Method 2: Using dcgain ??? DCgain = -SSG?
    poles2 = ctrl.poles(TF_2)
    TC_2 = -1 / poles2
    K2 = ctrl.dcgain(TF_2) # Method 2: Using dcgain ??? DCgain = -SSG?

    result.update({f"{r_types}" : {'TF_1':TF_1,'TF_2':TF_2,'TC_1':TC_1,'TC_2':TC_2, 'G1':K1, 'G2':K2}})



    #Flowchart Representation
    draw_tf_scheme(r_types, TF_1, TF_2, ctrl.tf(quotient.c,1))

  return result
    #Put the two configurations together
'''    if r_types == 'Serial':
      TF_r = ctrl.series(TF_1,TF_2)
    elif r_types == 'Parallel':
      TF_r = ctrl.parallel(TF_1,TF_2)
    elif r_types == 'Feedback':
      TF_r = ctrl.feedback(TF_1,TF_2)
    print(f"{r_types} conformation: {TF_r}")'''

    #2 - chronology















#if quotient == []:
#      print("No quotient")

  #den1 =  np.insert(-parameters_a, 0, 1)
  #den = [1] + (-parameters_a).tolist() #a [1,a2,a3,...] (endogenous)
  #TF = ctrl.TransferFunction(num, den)    #define transfer function














  #Step 4

for index, model in best_models_df.iterrows():
  A = model.loc['Parameters_a']
  B = model.loc['Parameters_b']
  print(f'\n\nModel: {model.loc["Code_Name"]} \nParameter "A": {A}\nParameters "B": {B}')

  print(decomposition_model(A,B))

p_b = np.array([0.0007, -0.0008])
p_a = np.array([1,-1.51104, 0.57089])
decomposition_model(p_a,p_b) # Q = [0.0004, 0.033798] || R = [0.0007, -0.0008]
'''
p_b = np.array([2,1]) # x +2
p_a = np.array([4,5,9,4]) #4x³+5x²+9x+4
decomposition_model(p_a,p_b) # Q = [4,1,3] || R = [-2]

#A(x) = 2x3 + 4x2 – 6x + 8 and B(x) = x2 + 3

p_b = np.array([2,4,-6,8])
p_a = np.array([1,0,3])
decomposition_model(p_a,p_b) # Q = [2,-2] || R = [0, 8]'''


p_b = np.array([0.6,0.3,0.54])
p_a = np.array([-0.56,-0.33])
result = decomposition_model(p_a,p_b)


In [ ]:
import math
import cmath

# Reverse function for Set 1
def serial(b_0,b_1, a_1, a_2): # Only considers 1 value for "B"
    # b'_{0-1} and b'_{0-2} can be any pair of numbers that multiply to b_0.
    #b_0_1 = math.sqrt(b_0)  # Assume a symmetric solution for simplicity
    b_0_1 = b_0 / 0.5
    b_0_2 = b_0 / b_0_1

    # Solve the quadratic equation for a'_{1-1} and a'_{1-2}
    discriminant = a_1**2 - 4 * a_2
    if discriminant < 0:
        print("Serial: No real solution exists for a_1 and a_2. Mathematically unfeasible Conformation.")
        a_1_1 = (a_1 + cmath.sqrt(discriminant)) / 2
        a_1_2 = (a_1 - cmath.sqrt(discriminant)) / 2
        return b_0_1, b_0_2, a_1_1, a_1_2
        #raise ValueError("No real solution exists for a_1 and a_2.")

    a_1_1 = (a_1 + math.sqrt(discriminant)) / 2
    a_1_2 = (a_1 - math.sqrt(discriminant)) / 2

    return b_0_1, b_0_2, a_1_1, a_1_2

# Reverse function for Set 2
def parallel(b_0, b_1, a_1, a_2):
    # Solve for a'_{1-1} and a'_{1-2} using the quadratic equation
    discriminant = a_1**2 - 4 * a_2
    if discriminant < 0:
        print("Parallel: No real solution exists for a_1 and a_2. Mathematically unfeasible Conformation.")
        a_1_1 = (a_1 + cmath.sqrt(discriminant)) / 2
        a_1_2 = (a_1 - cmath.sqrt(discriminant)) / 2
        #return -1
        #raise ValueError("No real solution exists for a_1 and a_2.")
    else:
      a_1_1 = (a_1 + math.sqrt(discriminant)) / 2
      a_1_2 = (a_1 - math.sqrt(discriminant)) / 2


    # Solve for b'_{0-2}
    if a_1_1 != a_1_2:  # Prevent division by zero
        print(a_1_1, a_1_2)
        b_0_2 = (b_1 - b_0 * a_1_2) / (a_1_1 - a_1_2)
    else:
        raise ValueError("a'_{1-1} and a'_{1-2} are equal, division by zero.")

    # Solve for b'_{0-1}
    b_0_1 = b_0 - b_0_2

    return b_0_1, b_0_2, a_1_1, a_1_2

# Reverse function for Set 3
def feedback(b_0, b_1, a_1, a_2):
    # Solve for the denominator (d)
    if b_1 != 0:
        d = b_0 / b_1
    else:
        d = 1 + b_0

    # Solve for b'_{0-1} and b'_{0-2}
    b_0_1 = b_0 * d
    if b_0_1 != 0:
        b_0_2 = (d - 1) / b_0_1
    else:
        b_0_2 = 0

    # Solve the quadratic equation for a'_{1-1} and a'_{1-2}
    discriminant = (a_1 * d)**2 - 4 * a_2 * d
    if discriminant < 0:
      print("Feedback: No real solution exists for a_1 and a_2. Mathematically unfeasible Conformation.")
      a_1_1 = (a_1 * d + cmath.sqrt(discriminant)) / 2
      a_1_2 = (a_1 * d - cmath.sqrt(discriminant)) / 2
      #raise ValueError("No real solution exists for a_1 and a_2.")
      #return -1
    else:
      a_1_1 = (a_1 * d + math.sqrt(discriminant)) / 2
      a_1_2 = (a_1 * d - math.sqrt(discriminant)) / 2

    return b_0_1, b_0_2, a_1_1, a_1_2

def feedback_m(b_0, b_1, a_1, a_2):
  a_1_2 = b_1 / b_0
  a_1_1 = (a_2*a_1_2) / (a_1*a_1_2 - a_2)

  d = (a_1_1 + a_1_2)/a_1

  b_0_1 = (a_1_1*a_1_2*b_0) / a_2
  b_0_2 = (((b_0_1 / b_0)-1)/b_0_1)
  b_0_2 = (a_2/b_1) - (1/b_0_1)
  return b_0_1, b_0_2, a_1_1, a_1_2

def reverse_set3(b_0, b_1, a_1, a_2):
    # Iteratively solve for b'_{0-2}
    def solve_b_0_2():
        # Start with an initial guess for b'_{0-2}
        #b_0_2 = -0.5
        for b_0_2 in [-0.5,2]:
          for _ in range(100):  # Iterate to refine the solution
              b_0_1 = b_0 / (1 - b_0 * b_0_2) if (1 - b_0 * b_0_2) != 0 else 0
              if b_0_1 == 0:
                  break
              denominator = 1 + b_0_1 * b_0_2
              a_1_1 = (a_1 * denominator + cmath.sqrt((a_1 * denominator)**2 - 4 * a_2 * denominator)) / 2
              a_1_2 = (a_1 * denominator - cmath.sqrt((a_1 * denominator)**2 - 4 * a_2 * denominator)) / 2
              predicted_b_1 = (b_0_1 * a_1_2) / denominator

              if abs(predicted_b_1 - b_1) < 1e-1:  # Converged
                  print("found")
                  return b_0_1, b_0_2, a_1_1, a_1_2
              b_0_2 *= 0.95  # Adjust guess slightly
          #return -1
          raise ValueError("Failed to converge on a solution for b'_{0-2}.")

    # Solve for b'_{0-2}, b'_{0-1}, a'_{1-1}, a'_{1-2}
    b_0_1, b_0_2, a_1_1, a_1_2 = solve_b_0_2()
    #solve_b_0_2()
    return b_0_1, b_0_2, a_1_1, a_1_2

def reverse_set31(b_0, b_1, a_1, a_2):
    # Step 1: Calculate z (b'_{0-1} * b'_{0-2})
    z = (1 / b_0) - 1

    # Step 2: Solve for a'_{1-1} and a'_{1-2}
    sum_a = a_1 * (1 + z)  # a'_{1-1} + a'_{1-2}
    prod_a = a_2 * (1 + z)  # a'_{1-1} * a'_{1-2}

    discriminant_a = sum_a**2 - 4 * prod_a
    if discriminant_a < 0:
        print("Feedback2: No real solution exists for a_1 and a_2. Mathematically unfeasible Conformation.")
        a_1_1 = (sum_a + cmath.sqrt(discriminant_a)) / 2
        a_1_2 = (sum_a - cmath.sqrt(discriminant_a)) / 2
        #raise ValueError("No real solution for a'_{1-1} and a'_{1-2}.")
        #return -1
    else:
      a_1_1 = (sum_a + math.sqrt(discriminant_a)) / 2
      a_1_2 = (sum_a - math.sqrt(discriminant_a)) / 2

    # Step 3: Solve for b'_{0-1}
    b_0_1 = (b_1 * (1 + z)) / a_1_2

    # Step 4: Solve for b'_{0-2}
    b_0_2 = z / b_0_1

    return b_0_1, b_0_2, a_1_1, a_1_2         #-B1,B2 and A1 >=> A2
#B0= 0.0007 B1= -0.0008, A1= -1.511, A2= 0.5708   ===>    b_1= -0.5393,  b_2= 0.54,  a_1= -0.7558,  a_2= -0.7552
print(serial(0.002375307035356264,-0.002295340647311899 ,1, -1.511))
print(parallel(0.002375307035356264,-0.002295340647311899 ,1, -1.511))
print(reverse_set3(0.002375307035356264,-0.002295340647311899 ,1, -1.511))

print(reverse_set31(0.002375307035356264,-0.002295340647311899 ,1, -1.511))
print(feedback(0.002375307035356264,-0.002295340647311899 ,1, -1.511))
print(feedback_m(0.002375307035356264,-0.002295340647311899 ,1, -1.511))

In [ ]:
# Test input values
b_0_1 = 0.002372
b_0_2 = -0.5
a_1_1 = 1.82553
a_1_2 = -0.82672

# Define the functions for the three sets
def set1(b_0_1, b_0_2, a_1_1, a_1_2): #Serial
    b_0 = b_0_1 * b_0_2
    b_1 = 0
    a_1 = a_1_1 + a_1_2
    a_2 = a_1_1 * a_1_2
    return b_0, b_1, a_1, a_2

def set2(b_0_1, b_0_2, a_1_1, a_1_2):#Parallel
    b_0 = b_0_1 + b_0_2
    b_1 = b_0_1 * a_1_2 + b_0_2 * a_1_1
    a_1 = a_1_1 + a_1_2
    a_2 = a_1_1 * a_1_2
    return b_0, b_1, a_1, a_2

def set3(b_0_1, b_0_2, a_1_1, a_1_2):#Feedback
    denominator = 1 + b_0_1 * b_0_2
    b_0 = b_0_1 / denominator
    b_1 = (b_0_1 * a_1_2) / denominator
    a_1 = (a_1_1 + a_1_2) / denominator
    a_2 = (a_1_1 * a_1_2) / denominator
    return b_0, b_1, a_1, a_2

# Calculate results for the test values
result_set33 = set3(0.0004072344486155723, 0.0331, -0.0501, 0.0185)
result_set1 = set1(b_0_1, b_0_2, a_1_1, a_1_2)
result_set2 = set2(b_0_1, b_0_2, a_1_1, a_1_2)
result_set3 = set3(b_0_1, b_0_2, a_1_1, a_1_2)
print("Set 1:", result_set1)
print("Set 2:", result_set2)
print("Set 3:", result_set3)
print("Personal:", result_set33)

In [ ]:
# Test output values
b_0_1 = -0.5393
b_0_2 = 0.54
a_1_1 = -0.7558
a_1_2 = -0.7552

tolerance=1e-3

#Calculate Inputs
result_set1 = set1(b_0_1, b_0_2, a_1_1, a_1_2)
result_set2 = set2(b_0_1, b_0_2, a_1_1, a_1_2)
result_set3 = set3(b_0_1, b_0_2, a_1_1, a_1_2)


#Verify if f(input) = output
b_0, b_1, a_1, a_2 = result_set1
s_b1, s_b2, s_a1, s_a2 = serial(b_0, b_1, a_1, a_2)
if (b_0_1, b_0_2, a_1_1, a_1_2) == (s_b1, s_b2, s_a1, s_a2):
  print("TRUEEEE S")
else:
  print("Series: ",s_b1, s_b2, s_a1, s_a2)

b_0, b_1, a_1, a_2 = result_set2
p_b1, p_b2, p_a1, p_a2 = parallel(b_0, b_1, a_1, a_2)

if all(np.isclose(v1, v2, atol=tolerance) for v1, v2 in zip([-b_0_1, -b_0_2, a_1_1, a_1_2], [p_b1, p_b2, p_a1, p_a2])):
  print("Values are CLOSE")

if (b_0_1, b_0_2, a_1_1, a_1_2) == (p_b1, p_b2, p_a1, p_a2):
  print("TRUEEEE P")
else:
  print("Parallel: ",p_b1, p_b2, p_a1, p_a2, "Result",b_0_1, b_0_2, a_1_1, a_1_2)


b_0, b_1, a_1, a_2 = result_set3
fr_b1, fr_b2, fr_a1, fr_a2 = feedback_m(b_0, b_1, a_1, a_2)
f_b1, f_b2, f_a1, f_a2 = feedback(b_0, b_1, a_1, a_2)
fr2_b1, fr2_b2, fr2_a1, fr2_a2 = reverse_set31(b_0, b_1, a_1, a_2)

if (b_0_1, b_0_2, a_1_1, a_1_2) == (f_b1, f_b2, f_a1, f_a2):
  print("Feedback1")
else:
  print("Feedback1: ",f_b1, f_b2, f_a1, f_a2)

if (b_0_1, b_0_2, a_1_1, a_1_2) == (fr_b1, fr_b2, fr_a1, fr_a2):
  print("Feedback2")
else:
  print("Feedback2: ",(fr_b1, fr_b2, fr_a1, fr_a2))

if (b_0_1, b_0_2, a_1_1, a_1_2) == (fr2_b1, fr2_b2, fr2_a1, fr2_a2):
  print("Feedbackr2")
else:
  print("Feedbackr2: ",fr2_b1, fr2_b2, fr2_a1, fr2_a2)



In [ ]:

p_b = np.array([0.0007, -0.0008])
p_a = np.array([1,-1.5110, 0.5708])

In [ ]:
# Test Input values
b_0 = 0.0007
b_1 = -0.0008
a_1 = -1.510
a_2 = 0.570

# Calculate Outputs

s_b1, s_b2, s_a1, s_a2 = serial(b_0, b_1, a_1, a_2)
result_set1 = set1(s_b1, s_b2, s_a1, s_a2)
if (b_0, b_1, a_1, a_2) == result_set1:
  print(f"Equal Serie: {result_set1}")
else:
  print("Series: ",result_set1)

p_b1, p_b2, p_a1, p_a2 = parallel(b_0, b_1, a_1, a_2)
result_set2 = set2(p_b1, p_b2, p_a1, p_a2)
if (b_0, b_1, a_1, a_2) == result_set2:
  print(f"\nEqual Parallel: {result_set2}\n{p_b1, p_b2, p_a1, p_a2}\n")
else:
  print("Parallel: ",result_set2,p_b1, p_b2, p_a1, p_a2,"\n\n")


fr_b1, fr_b2, fr_a1, fr_a2 = feedback_m(b_0, b_1, a_1, a_2)
result_set3 = set3(fr_b1, fr_b2, fr_a1, fr_a2)
if (b_0, b_1, a_1, a_2) == result_set3:
  print(f"\nEqual Feedback(mine): {result_set3}\n")
else:
  print("Feedback(mine): ",result_set3)

f_b1, f_b2, f_a1, f_a2 = feedback(b_0, b_1, a_1, a_2)
result_set3 = set3(f_b1, f_b2, f_a1, f_a2)
if (b_0, b_1, a_1, a_2) == result_set3:
  print(f"Equal Feedback(norm): {result_set3}")
else:
  print("Feedback(norm): ",result_set3)

fr2_b1, fr2_b2, fr2_a1, fr2_a2 = reverse_set31(b_0, b_1, a_1, a_2)
result_set3 = set3(fr2_b1, fr2_b2, fr2_a1, fr2_a2)
if (b_0, b_1, a_1, a_2) == result_set3:
  print(f"Equal Feedback(31): {result_set3}")
else:
  print("Feedback(31): ",result_set3)

try:
  fr1_b1, fr1_b2, fr1_a1, fr1_a2 = reverse_set3(b_0, b_1, a_1, a_2)
  result_set3 = set3(fr1_b1, fr1_b2, fr1_a1, fr1_a2)
except:
  print("No solution")
finally:
  if (b_0, b_1, a_1, a_2) == result_set3:
    print(f"Equal Feedback(iter): {result_set3}")
  else:
      print("Feedback(iter): ",result_set3)

In [ ]:
import networkx as nx
def draw_coupling_diagram(coupling_type, poles, residues):
    """
    Visualize the system as a block diagram for a given coupling type.

    Parameters:
    - coupling_type: str ("series", "parallel", or "feedback")
    - poles: List of poles
    - residues: List of residues
    """
    G = nx.DiGraph()

    # Nodes for each subsystem
    for i, (p, r) in enumerate(zip(poles, residues), 1):
        G.add_node(f"Subsystem {i}\nPole: {p:.2f}\nResidue: {r:.2f}")

    # Define connections based on coupling type
    if coupling_type == "series":
        edges = [(f"Subsystem {i}", f"Subsystem {i+1}") for i in range(1, len(poles))]
    elif coupling_type == "parallel":
        edges = [("Input", f"Subsystem {i}") for i in range(1, len(poles) + 1)]
        edges += [(f"Subsystem {i}", "Output") for i in range(1, len(poles) + 1)]
        G.add_node("Input")
        G.add_node("Output")
    elif coupling_type == "feedback":
        edges = [("Input", "Subsystem 1")]
        edges += [(f"Subsystem {i}", f"Subsystem {i+1}") for i in range(1, len(poles))]
        edges += [(f"Subsystem {len(poles)}", "Input")]

    # Add edges
    G.add_edges_from(edges)

    # Draw the graph
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G) if coupling_type != "parallel" else nx.shell_layout(G)
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_size=3000,
        node_color="lightblue",
        font_size=10,
        font_weight="bold",
        edge_color="gray",
    )
    plt.title(f"{coupling_type.capitalize()} Coupling Diagram", fontsize=14)
    plt.show()

# Example poles and residues (replace with real values from decomposition)
poles = [-1, -2, -3]
residues = [0.5, 0.3, 0.2]

# Visualize each coupling type
draw_coupling_diagram("series", poles, residues)
draw_coupling_diagram("parallel", poles, residues)
draw_coupling_diagram("feedback", poles, residues)

In [ ]:
#Using NumPy’s polydiv Function
A = [2, 4, -6, 8]
B = [1, 0, 3]

quotient, remainder = np.polydiv(A, B)

print(f"\nUsing NumPy’s polydiv Function\nQuotient: {quotient}\nRemainder: {remainder}\n")

#Using SymPy’s div Function
from sympy import symbols, div
x = symbols('x')
A = 2*x**3 + 4*x**2 - 6*x + 8
B = x**2 + 3

quotient, remainder = div(A, B, domain='QQ')

print(f"\nUsing SymPy’s div Function\nQuotient: {quotient}\nRemainder: {remainder}\n")

#Manual Polynomial Long Division
def poly_long_div(dividend, divisor):
    quotient = []
    while len(dividend) >= len(divisor):
        lead_coeff = dividend[0] / divisor[0]
        quotient.append(lead_coeff)
        dividend = [coeff - lead_coeff * div_coeff for coeff, div_coeff in zip(dividend, divisor + [0]*(len(dividend)-len(divisor)))]
        dividend.pop(0)
    return quotient, dividend

A = [2, 4, -6, 8]
B = [1, 0, 3]
quotient, remainder = poly_long_div(A, B)
print(f"\nManual Polynomial Long Division\nQuotient: {quotient}\nRemainder: {remainder}\n")

In [ ]:
def dataloader(data, na, nb, d):

    # Initialize empty arrays for lagged features
    u = data['u']
    X = pd.DataFrame()
    y = data['y']


    # Create lagged features for input ('b' parameters)
    for i in range(0,nb):
        X['u-{}'.format(i + d)] = u.shift(i + d)
    # Create lagged features for output ('a' parameters)
    for i in range(1,na+1):
        X['y-{}'.format(i)] = y.shift(i)


    X = X.iloc[max(na,nb+d):]
    # Set the target values
    y = y[max(na,nb+d):]

    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    return X, y